Cuando Alan Turing imaginó su famosa prueba de Turing⁠ en 1950, propuso una manera de evaluar la capacidad de una máquina para igualar la inteligencia humana. Podría haber probado muchas cosas, como la capacidad de reconocer a los gatos en imágenes, jugar al ajedrez, componer música o escapar de un laberinto, pero, curiosamente, eligió una tarea lingüística. Más específicamente, ideó un chatbot capaz de engañar a su interlocutor para que pensara que era humano.⁠ Esta prueba tiene sus debilidades: un conjunto de reglas codificadas puede engañar a humanos desprevenidos o ingenuos (por ejemplo, la máquina podría dar respuestas vagas predefinidas en respuesta a algunas palabras clave, podría fingir que está bromeando o borracha para obtener un pase en sus respuestas más extrañas, o podría escapar a las preguntas difíciles respondiéndoles con sus propias preguntas), y muchos aspectos de la inteligencia humana son completamente ignorados (por ejemplo, la capacidad de interpretar la comunicación no verbal, como las expresiones faciales, o de aprender un tarea manual). Pero la prueba destaca el hecho de que dominar el lenguaje es posiblemente la mayor capacidad cognitiva del Homo sapiens.

¿Podemos construir una máquina que pueda dominar el lenguaje escrito y hablado? Este es el objetivo final de la investigación de la PNL, pero es un poco demasiado amplio, por lo que en la práctica los investigadores se centran en tareas más específicas, como la clasificación de texto, la traducción, el resumen, la respuesta a preguntas y muchas más.

Un enfoque común para las tareas del lenguaje natural es utilizar redes neuronales recurrentes. Por lo tanto, continuaremos explorando los RNN (introducidos en el Capítulo 15), comenzando con un personaje RNN, o char-RNN, entrenado para predecir el siguiente personaje en una oración. Esto nos permitirá generar algún texto original. Primero usaremos un RNN sin estado (que aprende en porciones aleatorias de texto en cada iteración, sin ninguna información sobre el resto del texto), luego construiremos un RNN con estado (que preserva el estado oculto entre las iteraciones de entrenamiento y continúa leyendo donde lo dejó, lo que le permite aprender patrones más largos). A continuación, construiremos un RNN para realizar un análisis de sentimientos (por ejemplo, leer reseñas de películas y extraer el sentimiento del tasador sobre la película), esta vez tratando las oraciones como secuencias de palabras, en lugar de caracteres. Luego mostraremos cómo se pueden usar los RNN para construir una arquitectura de codificador-decodificador capaz de realizar la traducción automática neuronal (NMT), traduciendo del inglés al español.

En la segunda parte de este capítulo, exploraremos los mecanismos de atención. Como su nombre indica, estos son componentes de la red neuronal que aprenden a seleccionar la parte de las entradas en la que el resto del modelo debe centrarse en cada paso del tiempo. En primer lugar, aumentaremos el rendimiento de una arquitectura de codificador-decodificador basada en RNN utilizando la atención. A continuación, dejaremos caer los RNN por completo y utilizaremos una arquitectura de solo atención muy exitosa, llamada transformador, para construir un modelo de traducción. Luego discutiremos algunos de los avances más importantes en la PNL en los últimos años, incluidos modelos de lenguaje increíblemente potentes como GPT y BERT, ambos basados en transformadores. Por último, te mostraré cómo empezar con la excelente biblioteca de Transformers de Hugging Face.

Comencemos con un modelo simple y divertido que puede escribir como Shakespeare (más o menos).


# Generación de texto de Shakespeare usando un personaje RNN


En una famosa publicación de blog de 2015 titulada "La efectividad irrazonable de las redes neuronales recurrentes", Andrej Karpathy mostró cómo entrenar a un RNN para predecir el siguiente personaje en una frase. Este char-RNN se puede utilizar para generar texto novedoso, un carácter a la vez. Aquí hay una pequeña muestra del texto generado por un modelo char-RNN después de que fuera entrenado en todas las obras de Shakespeare:


    _PANDARUS:

    Por desgracia, creo que se le acercará y el día

    Cuando se lograría un poco de srain para no ser alimentado nunca,

    ¿Y quién es más que una cadena y sujetos de su muerte?

    No debería dormir._


No es exactamente una obra maestra, pero sigue siendo impresionante que el modelo fuera capaz de aprender palabras, gramática, puntuación adecuada y más, con solo aprender a predecir el siguiente carácter de una oración. Este es nuestro primer ejemplo de un modelo de lenguaje; modelos de lenguaje similares (pero mucho más poderosos), discutidos más adelante en este capítulo, están en el núcleo de la PNL moderna. En el resto de esta sección construiremos un char-RNN paso a paso, comenzando con la creación del conjunto de datos.


## Creación del conjunto de datos de formación



Primero, usando la práctica función `tf.keras.utils.get_file()` de Keras, descarguemos todas las obras de Shakespeare. Los datos se cargan desde el proyecto char-rnn de Andrej Karpathy:

In [1]:
import tensorflow as tf

shakespeare_url = "https://homl.info/shakespeare"  # shortcut URL
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

2024-03-24 18:52:14.908089: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1115394/1115394 [==============================] - 0s 0us/step


Vamos a imprimir las primeras líneas:

In [2]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


¡Parece que Shakespeare está bien!

A continuación, usaremos una capa `tf.keras.layers.TextVectorization` (introducida en el Capítulo 13) para codificar este texto. Configuramos `split="character"` para obtener codificación a nivel de caracteres en lugar de la codificación predeterminada a nivel de palabra, y usamos e `standarize="lower"` para convertir el texto a minúsculas (lo que simplificará la tarea):

In [3]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character",
                                                   standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

2024-03-24 18:53:42.290989: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Cada carácter ahora se asigna a un número entero, comenzando en 2. La capa `TextVectorization` reservó el valor 0 para tokens de relleno y reservó 1 para caracteres desconocidos. No necesitaremos ninguno de estos tokens por ahora, así que restemos 2 de los ID de los personajes y calculemos la cantidad de caracteres distintos y la cantidad total de caracteres:

In [4]:
encoded -= 2  # drop tokens 0 (pad) and 1 (unknown), which we will not use
n_tokens = text_vec_layer.vocabulary_size() - 2  # number of distinct chars = 39
dataset_size = len(encoded)  # total number of chars = 1,115,394

A continuación, al igual que hicimos en el capítulo 15, podemos convertir esta secuencia muy larga en un conjunto de datos de ventanas que luego podemos usar para entrenar un RNN de secuencia a secuencia. Los objetivos serán similares a los insumos, pero cambiarán un solo paso hacia el "futuro". Por ejemplo, una muestra en el conjunto de datos puede ser una secuencia de ID de caracteres que representan el texto "ser o no b" (sin la "e" final), y el objetivo correspondiente: una secuencia de ID de caracteres que representan el texto "o ser o no ser" (con la "e" final, pero sin la "t" principal). Escribamos una pequeña función de utilidad para convertir una larga secuencia de ID de caracteres en un conjunto de datos de pares de ventanas de entrada/destino:

In [5]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(buffer_size=100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

Esta función comienza de manera muy similar a la función de utilidad personalizada `to_windows()` que creamos en el Capítulo 15:

- Toma una secuencia como entrada (es decir, el texto codificado) y crea un conjunto de datos que contiene todas las ventanas de la longitud deseada.

* Aumenta la longitud en uno, ya que necesitamos el siguiente personaje para el objetivo.

- Luego, baraja las ventanas (opcionalmente), las agrupa, las divide en pares de entrada/salida y activa la precarga.

La figura 16-1 resume los pasos de preparación del conjunto de datos: muestra ventanas de longitud 11 y un tamaño de lote de 3. El índice de inicio de cada ventana se indica junto a él.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1601.png)

(_Figura 16-1. Preparación de un conjunto de datos de ventanas barajadas_)


Ahora estamos listos para crear el conjunto de entrenamiento, el conjunto de validación y el conjunto de pruebas. Utilizaremos aproximadamente el 90 % del texto para la capacitación, el 5 % para la validación y el 5 % para las pruebas:

In [6]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True,
                       seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

#### TIP

Establecemos la longitud de la ventana en 100, pero puedes intentar ajustarla: es más fácil y rápido entrenar a los RNN en secuencias de entrada más cortas, pero el RNN no podrá aprender ningún patrón más largo que length, así que no lo hagas demasiado pequeño.

#### -------------------------------------------------------------------------------------

¡Eso es todo! Preparar el conjunto de datos fue la parte más difícil. Ahora vamos a crear el modelo.


## Construcción y formación del modelo Char-RNN

Dado que nuestro conjunto de datos es razonablemente grande y el lenguaje de modelado es una tarea bastante difícil, necesitamos más que un simple RNN con algunas neuronas recurrentes. Construyamos y entrenemos un modelo con una capa `GRU` compuesta por 128 unidades (puede intentar ajustar la cantidad de capas y unidades más adelante, si es necesario):

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True)
history = model.fit(train_set, validation_data=valid_set, epochs=10,
                    callbacks=[model_ckpt])

Epoch 1/10


2024-03-24 19:54:26.479142: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 98474 of 100000
2024-03-24 19:54:26.636823: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


   1834/Unknown - 98s 47ms/step - loss: 1.9872 - accuracy: 0.4193

KeyboardInterrupt: 

Vamos a repasar este código:

* Usamos una capa de `Embedding` como primera capa para codificar las ID de los caracteres (las incrustaciones se introdujeron en el Capítulo 13). El número de dimensiones de entrada de la capa de `Embedding` es el número de ID de caracteres distintos, y el número de dimensiones de salida es un hiperparámetro que puede ajustar; lo configuraremos en 16 por ahora. Mientras que las entradas de la capa de `Embedding` serán tensores de forma 2D [tamaño de lote, longitud de ventana], la salida de la capa de `Embedding` será un tensor de forma 3D [tamaño de lote, longitud de ventana, tamaño de incrustación].

- Usamos una capa `Dense` para la capa de salida: debe tener 39 unidades (`n_tokens`) porque hay 39 caracteres distintos en el texto y queremos generar una probabilidad para cada carácter posible (en cada paso de tiempo). Las 39 probabilidades de salida deben sumar 1 en cada paso de tiempo, por lo que aplicamos la función de activación softmax a las salidas de la capa `Dense`.

* Por último, compilamos este modelo usando la pérdida `"sparse_categorical_crossentropy"` y un optimizador Nadam, y entrenamos el modelo para varias épocas,3 usando una devolución de llamada `ModelCheckpoint` para guardar el mejor modelo (en términos de precisión de validación) a medida que avanza el entrenamiento.


#### PROPINA

Si está ejecutando este código en Colab con una GPU activada, el entrenamiento debería durar aproximadamente de una a dos horas. Puedes reducir el número de épocas si no quieres esperar tanto tiempo, pero, por supuesto, la precisión del modelo probablemente será menor. Si se agota el tiempo de espera de la sesión de Colab, asegúrese de volver a conectarse rápidamente, o de lo contrario se destruirá el tiempo de ejecución de Colab.

#### -------------------------------------------------------------------------------------

Este modelo no maneja el preprocesamiento de texto, así que vamos a incluirlo en un modelo final que contenga la capa `tf.keras.layers.TextVectorization` como primera capa, más una capa `tf.keras.layers.Lambda` para restar 2 de los ID de los caracteres, ya que No estamos usando el relleno ni los tokens desconocidos por ahora:

In [ ]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    model
])

Y ahora vamos a usarlo para predecir el siguiente personaje en una oración:

In [ ]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)  # choose the most probable character ID
text_vec_layer.get_vocabulary()[y_pred + 2]

'''e'''

Genial, el modelo predijo correctamente el siguiente personaje. ¡Ahora usemos este modelo para fingir que somos Shakespeare!


## Generación de texto falso de Shakespeare


Para generar texto nuevo usando el modelo char-RNN, podríamos alimentarlo con algo de texto, hacer que el modelo prediga la siguiente letra más probable, agregarlo al final del texto y luego darle el texto extendido al modelo para que adivine la siguiente letra. , etcétera. A esto se le llama decodificación codiciosa. Pero en la práctica esto a menudo lleva a que las mismas palabras se repitan una y otra vez. En cambio, podemos muestrear el siguiente carácter aleatoriamente, con una probabilidad igual a la probabilidad estimada, usando la función `tf.random.categorical()` de TensorFlow. Esto generará un texto más diverso e interesante. La función `categorical()` muestra índices de clase aleatorios, dadas las probabilidades logarítmicas de clase (logits). Por ejemplo:



In [ ]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])  # probas = 50%, 40%, and 10%
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)  # draw 8 samples

Para tener más control sobre la diversidad del texto generado, podemos dividir los logits por un número llamado temperatura, que podemos modificar como queramos. Una temperatura cercana a cero favorece a los personajes de alta probabilidad, mientras que una temperatura alta da a todos los personajes la misma probabilidad. Por lo general, se prefieren temperaturas más bajas cuando se genera texto bastante rígido y preciso, como ecuaciones matemáticas, mientras que se prefieren temperaturas más altas cuando se genera texto más diverso y creativo. La siguiente función auxiliar personalizada `next_char()` utiliza este enfoque para seleccionar el siguiente carácter para agregar al texto de entrada:

In [ ]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

A continuación, podemos escribir otra pequeña función auxiliar que llamará repetidamente a `next_char()` para obtener el siguiente carácter y agregarlo al texto dado: 

In [ ]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

¡Ahora estamos listos para generar algo de texto! Intentemos con diferentes valores de temperatura:

In [ ]:
tf.random.set_seed(42)
print(extend_text("To be or not to be", temperature=0.01))

'''
To be or not to be the duke
as it is a proper strange death,
and the
'''

print(extend_text("To be or not to be", temperature=1))

'''
To be or not to behold?
second push:
gremio, lord all, a sistermen,
'''

print(extend_text("To be or not to be", temperature=100))

'''
To be or not to bef ,mt'&o3fpadm!$
wh!nse?bws3est--vgerdjw?c-y-ewznq
'''

Shakespeare parece estar sufriendo una ola de calor. Para generar texto más convincente, una técnica común es tomar muestras solo de los k caracteres superiores, o solo del conjunto más pequeño de caracteres superiores cuya probabilidad total exceda algún umbral (esto se llama muestreo de núcleo). Alternativamente, puede intentar usar la búsqueda por haz, que discutiremos más adelante en este capítulo, o usar más capas `GRU` y más neuronas por capa, entrenar durante más tiempo y agregar algo de regularización si es necesario. También tenga en cuenta que el modelo actualmente es incapaz de aprender patrones de más de 100 caracteres. Podría intentar agrandar esta ventana, pero también hará que el entrenamiento sea más difícil, e incluso las células LSTM y GRU no pueden manejar secuencias muy largas. Un enfoque alternativo es utilizar un RNN con estado.


## RNN con estado


Hasta ahora, solo hemos utilizado RNN sin estado: en cada iteración de entrenamiento, el modelo comienza con un estado oculto lleno de ceros, luego actualiza este estado en cada paso de tiempo, y después del último paso de tiempo, lo tira, ya que ya no es necesario. ¿Qué pasaría si le diéramos instrucciones al RNN para que preservara este estado final después de procesar un lote de entrenamiento y lo usáramos como estado inicial para el siguiente lote de entrenamiento? De esta manera, el modelo podría aprender patrones a largo plazo a pesar de solo propagarse a través de secuencias cortas. Esto se llama astateful RNN. Vamos a repasar cómo construir uno.

Primero, tenga en cuenta que un RNN con estado solo tiene sentido si cada secuencia de entrada en un lote comienza exactamente donde terminó la secuencia correspondiente en el lote anterior. Entonces, lo primero que debemos hacer para construir un RNN con estado es usar secuencias de entrada secuenciales y no superpuestas (en lugar de las secuencias mezcladas y superpuestas que usamos para entrenar RNN sin estado). Por lo tanto, al crear `tf.data.Dataset`, debemos usar `shift=length` (en lugar de `shift=1`) al llamar al método `window()`. Además, no debemos llamar al método `shuffle()`.

Desafortunadamente, el procesamiento por lotes es mucho más difícil cuando se prepara un conjunto de datos para un RNN con estado que para un RNN sin estado. De hecho, si llamaramos al `batch(32)`, entonces se colocarían 32 ventanas consecutivas en el mismo lote y el siguiente lote no continuaría con cada una de estas ventanas donde lo dejó. El primer lote contendría las ventanas 1 a 32 y el segundo lote contendría las ventanas 33 a 64, por lo que si considera, digamos, la primera ventana de cada lote (es decir, las ventanas 1 y 33), puede ver que no son consecutivas. . La solución más sencilla a este problema es utilizar simplemente un tamaño de lote de 1. La siguiente función de utilidad personalizada `to_dataset_for_stateful_rnn()` utiliza esta estrategia para preparar un conjunto de datos para un RNN con estado:

In [ ]:
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length + 1)).batch(1)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

stateful_train_set = to_dataset_for_stateful_rnn(encoded[:1_000_000], length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000],
                                                 length)
stateful_test_set = to_dataset_for_stateful_rnn(encoded[1_060_000:], length)

La figura 16-2 resume los pasos principales de esta función.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1602.png)

(_Figura 16-2. Preparación de un conjunto de datos de fragmentos de secuencias consecutivas para un RNN con estado_)


El procesamiento por lotes es más difícil, pero no imposible. Por ejemplo, podríamos dividir el texto de Shakespeare en 32 textos de igual longitud, crear un conjunto de datos de secuencias de entrada consecutivas para cada uno de ellos y, finalmente, usar `tf.data.Dataset.zip(datasets).map(lambda *windows: tf.stack (windows))` para crear lotes consecutivos adecuados, donde la enésima secuencia de entrada de un lote comienza exactamente donde n^(th) la enésima secuencia de entrada en el lote anterior (consulte el cuaderno para obtener el código completo).

Ahora, creemos el RNN con estado. Necesitamos establecer el argumento `stateful` en True al crear cada capa recurrente, y porque el RNN con estado necesita conocer el tamaño del lote (ya que preservará un estado para cada secuencia de entrada en el lote). Por lo tanto, debemos establecer el argumento `batch_input_shape` en la primera capa. Tenga en cuenta que podemos dejar la segunda dimensión sin especificar, ya que las secuencias de entrada podrían tener cualquier longitud:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16,
                              batch_input_shape=[1, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

Al final de cada época, necesitamos restablecer los estados antes de volver al principio del texto. Para esto, podemos usar una pequeña devolución de llamada de Keras personalizada:

In [ ]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

Y ahora podemos compilar el modelo y entrenarlo usando nuestra devolución de llamada:

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(stateful_train_set, validation_data=stateful_valid_set,
                    epochs=10, callbacks=[ResetStatesCallback(), model_ckpt])

#### TIP

Después de entrenar este modelo, solo será posible usarlo para hacer predicciones para lotes del mismo tamaño que se utilizaron durante el entrenamiento. Para evitar esta restricción, cree un modelo sin estado idéntico y copie los pesos del modelo con estado en este modelo.

#### --------------------------------------------------------------------------------------

Curiosamente, aunque un modelo de char-RNN solo está entrenado para predecir el siguiente personaje, esta tarea aparentemente simple en realidad también requiere que aprenda algunas tareas de nivel superior. Por ejemplo, para encontrar el siguiente personaje después de "Gran película, de verdad", es útil entender que la oración es positiva, por lo que lo que sigue es más probable que sea la letra "l" (para "amado") en lugar de "h" (para "odiado"). De hecho, un artículo de 2017⁠ de Alec Radford y otros investigadores de OpenAI describe cómo los autores entrenaron un gran modelo similar a char-RNN en un gran conjunto de datos, y encontraron que una de las neuronas actuó como un excelente clasificador de análisis de sentimientos: aunque el modelo fue entrenado sin ninguna etiqueta, la neurona de sentimiento, como la llamaron, alcanzó un rendimiento de última generación en los puntos de referencia de análisis de sentimientos. Esto presagiaba y motivaba el preentrenamiento no supervisado en PNL.

Pero antes de explorar el preentrenamiento no supervisado, centremos nuestra atención en los modelos a nivel de palabra y cómo usarlos de manera supervisada para el análisis de sentimientos. En el proceso, aprenderás a manejar secuencias de longitudes variables usando el enmascaramiento.


# Análisis de sentimientos


Generar texto puede ser divertido e instructivo, pero en los proyectos de la vida real, una de las aplicaciones más comunes de la PNL es la clasificación de texto, especialmente el análisis de sentimientos. Si la clasificación de imágenes en el conjunto de datos de MNIST es "¡Hola mundo!" de la visión por ordenador, luego el análisis de sentimientos en el conjunto de datos de reseñas de IMDb es el "¡Hola mundo!" de procesamiento del lenguaje natural. El conjunto de datos de IMDb consta de 50 000 reseñas de películas en inglés (25 000 para capacitación, 25 000 para pruebas) extraídas de la famosa base de datos de películas de Internet, junto con un objetivo binario simple para cada revisión que indica si es negativa (0) o positiva (1). Al igual que el MNIST, el conjunto de datos de reseñas de IMDb es popular por buenas razones: es lo suficientemente simple como para ser abordado en una computadora portátil en un período de tiempo razonable, pero lo suficientemente desafiante como para ser divertido y gratificante.

Vamos a cargar el conjunto de datos de IMDb utilizando la biblioteca de conjuntos de datos de TensorFlow (introducida en el capítulo 13). Utilizaremos el primer 90 % del conjunto de entrenamiento para el entrenamiento, y el 10 % restante para la validación:

In [ ]:
import tensorflow_datasets as tfds

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "train[90%:]", "test"],
    as_supervised=True
)
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

#### TIP

Keras también incluye una función para cargar el conjunto de datos de IMDb, si lo prefiere: `tf.keras.datasets.imdb.load_data()`. Las revisiones ya están preprocesadas como secuencias de ID de palabras.

#### -------------------------------------------------------------------------------------

Vamos a inspeccionar algunas reseñas:

In [ ]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode("utf-8"))
    print("Label:", label.numpy())

'''
This was an absolutely terrible movie. Don't be lured in by Christopher [...]
Label: 0
I have been known to fall asleep during films, but this is usually due to [...]
Label: 0
Mann photographs the Alberta Rocky Mountains in a superb fashion, and [...]
Label: 0
This is the kind of film for a snowy Sunday afternoon when the rest of the [...]
Label: 1
'''

Algunas reseñas son fáciles de clasificar. Por ejemplo, la primera reseña incluye las palabras "película terrible" en la primera frase. Pero en muchos casos las cosas no son tan simples. Por ejemplo, la tercera revisión comienza positivamente, a pesar de que en última instancia es una revisión negativa (etiqueta 0).

Para construir un modelo para esta tarea, necesitamos preprocesar el texto, pero esta vez lo cortaremos en palabras en lugar de caracteres. Para esto, podemos usar la capa `tf.keras.lay⁠ers.TextVectorization` nuevamente. Tenga en cuenta que utiliza espacios para identificar los límites de las palabras, lo que no funcionará bien en algunos idiomas. Por ejemplo, la escritura china no usa espacios entre palabras, la vietnamita usa espacios incluso dentro de las palabras y el alemán a menudo une varias palabras juntas, sin espacios. Incluso en inglés, los espacios no siempre son la mejor manera de tokenizar el texto: piense en "San Francisco" o "#ILoveDeepLearning".

Afortunadamente, hay soluciones para abordar estos problemas. En un artículo de 2016, Rico Sennrich et al. de la Universidad de Edimburgo exploraron varios métodos para tokenizar y destokenizar el texto a nivel de subpalabra. De esta manera, incluso si su modelo se encuentra con una palabra rara que nunca ha visto antes, todavía puede adivinar razonablemente lo que significa. Por ejemplo, incluso si el modelo nunca vio la palabra "más inteligente" durante el entrenamiento, si aprendió la palabra "inteligente" y también aprendió que el sufijo "est" significa "más", puede inferir el significado de "más inteligente". Una de las técnicas que evaluaron los autores es la codificación de pares de bytes (BPE). BPE funciona dividiendo todo el conjunto de entrenamiento en caracteres individuales (incluidos los espacios), y luego fusionando repetidamente los pares adyacentes más frecuentes hasta que el vocabulario alcance el tamaño deseado.

Un documento⁠ de 2018 de Taku Kudo en Google mejoró aún más la tokenización de subpalabras, a menudo eliminando la necesidad de un preprocesamiento específico del idioma antes de la tokenización. Además, el documento propuso una nueva técnica de regularización llamada regularización de subpalabras, que mejora la precisión y la robustez al introducir cierta aleatoriedad en la tokenización durante el entrenamiento: por ejemplo, "Nueva Inglaterra" puede ser tokenizada como "Nuevo" + "Inglaterra", o "Nuevo" + "Ing" + "tierra", o simplemente "Nueva Inglaterra" (solo un token). El proyecto SentencePiece de Google proporciona una implementación de código abierto, que se describe en un documento⁠ de Taku Kudo y John Richardson.

La biblioteca de texto TensorFlow también implementa varias estrategias de tokenización, incluyendo WordPiece⁠ (una variante de BPE) y por último, pero no menos importante, la biblioteca de tokens de Hugging Face implementa una amplia gama de tokenizadores extremadamente rápidos.

Sin embargo, para la tarea de IMDb en inglés, usar espacios para los límites de los tokens debería ser suficiente. Entonces, sigamos adelante con la creación de una capa `TextVectorization` y adaptémosla al conjunto de entrenamiento. Limitaremos el vocabulario a 1000 tokens, incluidas las 998 palabras más frecuentes más un token de relleno y un token para palabras desconocidas, ya que es poco probable que palabras muy raras sean importantes para esta tarea, y limitar el tamaño del vocabulario reducirá el número de Parámetros que el modelo necesita aprender:

In [ ]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

Finalmente, podemos crear el modelo y entrenarlo:

In [ ]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

La primera capa es la capa `TextVectorization` que acabamos de preparar, seguida de una capa de `Embedding` que convertirá los ID de palabras en incrustaciones. La matriz de incrustación debe tener una fila por token en el vocabulario (`vocab_size`) y una columna por dimensión de incrustación (este ejemplo usa 128 dimensiones, pero este es un hiperparámetro que puede ajustar). A continuación usamos una capa `GRU` y una capa `Dense` con una sola neurona y la función de activación sigmoidea, ya que esta es una tarea de clasificación binaria: la salida del modelo será la probabilidad estimada de que la reseña exprese un sentimiento positivo con respecto a la película. Luego compilamos el modelo y lo ajustamos al conjunto de datos que preparamos anteriormente durante un par de épocas (o puedes entrenar durante más tiempo para obtener mejores resultados).

Lamentablemente, si ejecuta este código, generalmente encontrará que el modelo no aprende nada en absoluto: la precisión se mantiene cerca del 50%, no mejor que la probabilidad aleatoria. ¿Porqué es eso? Las revisiones tienen diferentes longitudes, por lo que cuando la capa `TextVectorization` las convierte en secuencias de ID de token, rellena las secuencias más cortas usando el token de relleno (con ID 0) para que sean tan largas como la secuencia más larga del lote. Como resultado, la mayoría de las secuencias terminan con muchos tokens de relleno, a menudo docenas o incluso cientos de ellos. Aunque estamos usando una capa `GRU`, que es mucho mejor que una capa `SimpleRNN`, su memoria a corto plazo aún no es excelente, por lo que cuando pasa por muchos tokens de relleno, ¡termina olvidando de qué se trataba la revisión! Una solución es alimentar el modelo con lotes de oraciones de igual longitud (lo que también acelera el entrenamiento). Otra solución es hacer que RNN ignore los tokens de relleno. Esto se puede hacer usando enmascaramiento.


## Enmascaramiento

Hacer que el modelo ignore los tokens de relleno es trivial usando Keras: simplemente agregue `mask_zero=True` al crear la capa de `Embedding`. Esto significa que todas las capas posteriores ignorarán los tokens de relleno (cuyo ID es 0). ¡Eso es todo! Si vuelve a entrenar el modelo anterior durante algunas épocas, encontrará que la precisión de la validación alcanza rápidamente más del 80%.

La forma en que esto funciona es que la capa de `Embedding` crea un tensor de máscara igual a `tf.math.not_equal(inputs, 0):` es un tensor booleano con la misma forma que las entradas y es igual a `False` en cualquier lugar donde se encuentren los ID de los tokens. 0, o `True` en caso contrario. Luego, el modelo propaga automáticamente este tensor de máscara a la siguiente capa. Si el método `call()` de esa capa tiene un argumento de `mask`, automáticamente recibe la máscara. Esto permite que la capa ignore los pasos de tiempo apropiados. Cada capa puede manejar la máscara de manera diferente, pero en general simplemente ignoran los pasos de tiempo enmascarados (es decir, los pasos de tiempo para los cuales la máscara es `False`). Por ejemplo, cuando una capa recurrente encuentra un paso de tiempo enmascarado, simplemente copia la salida del paso de tiempo anterior.

A continuación, si el atributo `supports_masking` de la capa es `True`, la máscara se propaga automáticamente a la siguiente capa. Sigue propagándose de esta manera mientras las capas tengan `support_masking=True`. Como ejemplo, el atributo `support_mask⁠ing` de una capa recurrente es `True` cuando `return_sequences=True`, pero es `False` cuando `return_sequen⁠ces=False` ya que en este caso ya no es necesaria una máscara. Entonces, si tiene un modelo con varias capas recurrentes con `return_sequences=True`, seguidas de una capa recurrente con `return_sequences=False`, entonces la máscara se propagará automáticamente hasta la última capa recurrente: esa capa usará la máscara para ignorar los pasos enmascarados, pero no propagará más la máscara. De manera similar, si configura `mask_zero=True` al crear la capa de `Embedding` en el modelo de análisis de sentimiento que acabamos de crear, entonces la capa `GRU` recibirá y usará la máscara automáticamente, pero no la propagará más, ya que `return_sequences` no está configurado en `True`.

#### TIP

Algunas capas necesitan actualizar la máscara antes de propagarla a la siguiente capa: lo hacen implementando el método `compute_mask()`, que toma dos argumentos: las entradas y la máscara anterior. Luego calcula la máscara actualizada y la devuelve. La implementación predeterminada de `compute_mask()` simplemente devuelve la máscara anterior sin cambios.

#### -------------------------------------------------------------------------------------

Muchas capas de Keras admiten enmascaramiento: `SimpleRNN, GRU, LSTM, Bidireccional, Dense, TimeDistributed, Add` y algunas otras (todas en el paquete `tf.keras.layers`). Sin embargo, las capas convolucionales (incluida `Conv1D`) no admiten el enmascaramiento; de todos modos, no es obvio cómo lo harían.

Si la máscara se propaga hasta la salida, entonces también se aplica a las pérdidas, por lo que los pasos de tiempo enmascarados no contribuirán a la pérdida (su pérdida será de 0). Esto supone que el modelo genera secuencias, lo que no es el caso en nuestro modelo de análisis de sentimientos.


#### ADVERTENCIA

Las capas `LSTM` y `GRU` tienen una implementación optimizada para GPU, basada en la biblioteca cuDNN de Nvidia. Sin embargo, esta implementación solo admite el enmascaramiento si todos los tokens de relleno están al final de las secuencias. También requiere que utilice los valores predeterminados para varios hiperparámetros: `activation, recurrent_activation, recurrent_dropout, unroll, use_bias y reset_after`. Si ese no es el caso, estas capas volverán a la implementación de GPU predeterminada (mucho más lenta).

#### --------------------------------------------------------------------------------------

Si desea implementar su propia capa personalizada con soporte de `mask`, debe agregar un argumento de máscara al método `call()` y, obviamente, hacer que el método use la máscara. Además, si la máscara debe propagarse a las siguientes capas, entonces debes configurar `self.supports_masking=True` en el constructor. Si la máscara debe actualizarse antes de propagarse, debe implementar el método `compute_mask()`.

Si su modelo no comienza con una capa de `Embedding`, puede usar la capa `tf.keras.layers.Masking`: de forma predeterminada, establece la máscara en `tf.math.reduce_any(tf.math.not_equal(X , 0), axis=-1)`, lo que significa que los pasos de tiempo en los que la última dimensión está llena de ceros se enmascararán en capas posteriores.

El uso de capas de máscara y la propagación automática de máscaras funciona mejor para modelos simples. No siempre funcionará para modelos más complejos, como cuando necesitas mezclar capas `Conv1D` con capas recurrentes. En tales casos, deberá calcular explícitamente la máscara y pasarla a las capas apropiadas, utilizando la API funcional o la API de subclasificación. Por ejemplo, el siguiente modelo es equivalente al modelo anterior, excepto que se crea utilizando la API funcional y maneja el enmascaramiento manualmente. También añade un poco de abandono ya que el modelo anterior estaba ligeramente sobreajustado:

In [ ]:
inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
token_ids = text_vec_layer(inputs)
mask = tf.math.not_equal(token_ids, 0)
Z = tf.keras.layers.Embedding(vocab_size, embed_size)(token_ids)
Z = tf.keras.layers.GRU(128, dropout=0.2)(Z, mask=mask)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(Z)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

Un último enfoque para el enmascaramiento es alimentar el modelo con tensores irregulares.⁠ En la práctica, todo lo que necesita hacer es establecer `ragged=True` al crear la capa `TextVectorization`, de modo que las secuencias de entrada se representen como tensores irregulares:

In [ ]:
text_vec_layer_ragged = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size, ragged=True)

text_vec_layer_ragged.adapt(train_set.map(lambda reviews, labels: reviews))
text_vec_layer_ragged(["Great movie!", "This is DiCaprio's best role."])

'''
<tf.RaggedTensor [[86, 18], [11, 7, 1, 116, 217]]>
'''

Compare esta representación de tensor irregular con la representación de tensor regular, que utiliza tokens de relleno:

In [ ]:
text_vec_layer(["Great movie!", "This is DiCaprio's best role."])

'''
<tf.Tensor: shape=(2, 5), dtype=int64, numpy=
array([[ 86,  18,   0,   0,   0],
       [ 11,   7,   1, 116, 217]])>
'''

Las capas recurrentes de Keras tienen soporte incorporado para tensores irregulares, por lo que no necesita hacer nada más: simplemente use esta capa `TextVectorization` en su modelo. No es necesario pasar `mask_zero=True` ni manejar máscaras explícitamente: todo está implementado para usted. ¡Eso es conveniente! Sin embargo, a principios de 2022, el soporte para tensores irregulares en Keras todavía es bastante reciente, por lo que existen algunas asperezas. Por ejemplo, actualmente no es posible utilizar tensores irregulares como objetivos cuando se ejecuta en la GPU (pero esto puede resolverse cuando lea estas líneas).

Sea cual sea el enfoque de enmascaramiento que prefieras, después de entrenar este modelo durante algunas épocas, será bastante bueno para juzgar si una revisión es positiva o no. Si utiliza la devolución de llamada `tf.keras.callbacks.TensorBoard()`, puede visualizar las incrustaciones en TensorBoard a medida que se aprenden: es fascinante ver palabras como "impresionante" y "increíble" que se agrupan gradualmente en un lado del espacio de incrustación, mientras que palabras como "horrible" y "terrible" se agrupan en el otro lado. Algunas palabras no son tan positivas como cabría esperar (al menos con este modelo), como la palabra "bueno", presumiblemente porque muchas críticas negativas contienen la frase "no bueno".


## Reutilización de incrustaciones preentrenadas y modelos de lenguaje


Es impresionante que el modelo sea capaz de aprender incrustaciones de palabras útiles basadas en solo 25 000 reseñas de películas. ¡Imagina lo buenas que serían las incrustaciones si tuviéramos miles de millones de reseñas en las que entrenar! Desafortunadamente, no lo hacemos, pero tal vez podamos reutilizar incrustaciones de palabras entrenadas en algún otro corpus de texto (muy) grande (por ejemplo, reseñas de Amazon, disponibles en TensorFlow Datasets), incluso si no está compuesto de reseñas de películas. Después de todo, la palabra "increíble" generalmente tiene el mismo significado, ya sea que la uses para hablar de películas o cualquier otra cosa. Además, tal vez las incrustaciones serían útiles para el análisis de sentimientos, incluso si estuvieran entrenadas en otra tarea: dado que palabras como "impresionante" y "increíble" tienen un significado similar, es probable que se agrupen en el espacio de incrustación incluso para tareas como predecir la siguiente palabra en una oración. Si todas las palabras positivas y todas las palabras negativas forman grupos, entonces esto será útil para el análisis de sentimientos. Por lo tanto, en lugar de entrenar incrustaciones de palabras, podríamos descargar y usar incrustaciones preentrenadas, como las incrustaciones Word2vec de Google, las incrustaciones GloVe de Stanford o las incrustaciones FastText de Facebook.

El uso de incrustaciones de palabras preentrenadas fue popular durante varios años, pero este enfoque tiene sus límites. En particular, una palabra tiene una sola representación, sin importar el contexto. Por ejemplo, la palabra "derecha" se codifica de la misma manera en "izquierda y derecha" y "derecha e incorrecta", a pesar de que significa dos cosas muy diferentes. Para abordar esta limitación, un documento de 2018⁠ de Matthew Peters introdujo incrustaciones de modelos de lenguaje (ELMo): estas son incrustaciones de palabras contextualizadas aprendidas de los estados internos de un modelo de lenguaje bidireccional profundo. En lugar de solo usar incrustaciones preentrenadas en su modelo, reutiliza parte de un modelo de lenguaje preentrenado.

Más o menos al mismo tiempo, el documento Universal Language Model Fine-Tuning (ULMFiT)⁠ de Jeremy Howard y Sebastian Ruder demostró la eficacia del preentrenamiento no supervisado para las tareas de PNL: los autores entrenaron un modelo de lenguaje LSTM en un enorme corpus de texto utilizando el aprendizaje autosupervisado (es decir, generando las etiquetas automáticamente a partir de los datos), luego lo ajustaron en varias tareas. Su modelo superó al estado de la técnica en seis tareas de clasificación de texto por un gran margen (reduciendo la tasa de error en un 18-24% en la mayoría de los casos). Además, los autores mostraron que un modelo preentrenado ajustado en solo 100 ejemplos etiquetados podría lograr el mismo rendimiento que uno entrenado desde cero en 10.000 ejemplos. Antes del documento ULMFiT, el uso de modelos preentrenados era solo la norma en la visión por ordenador; en el contexto de la PNL, el preentrenamiento se limitaba a las incrustaciones de palabras. Este documento marcó el comienzo de una nueva era en la PNL: hoy en día, la reutilización de modelos lingüísticos preentrenados es la norma.

Por ejemplo, construyamos un clasificador basado en el Codificador de Sentencias Universal, una arquitectura de modelo introducida en un documento de 2018⁠ por un equipo de investigadores de Google. Este modelo se basa en la arquitectura del transformador, que veremos más adelante en este capítulo. Convenientemente, el modelo está disponible en TensorFlow Hub:

In [ ]:
import os
import tensorflow_hub as hub

os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                   trainable=True, dtype=tf.string, input_shape=[]),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, validation_data=valid_set, epochs=10)

#### TIP

Este modelo es bastante grande (cerca de 1 GB), por lo que la descarga puede tardar un poco. De forma predeterminada, los módulos de TensorFlow Hub se guardan en un directorio temporal y se descargan una y otra vez cada vez que ejecuta su programa. Para evitarlo, debe configurar la variable de entorno `TFHUB_CACHE_DIR` en un directorio de su elección: los módulos se guardarán allí y solo se descargarán una vez.

#### --------------------------------------------------------------------------------------

Tenga en cuenta que la última parte de la URL del módulo TensorFlow Hub especifica que queremos la versión 4 del modelo. Este control de versiones garantiza que si se lanza una nueva versión del módulo en TF Hub, no romperá nuestro modelo. Convenientemente, si solo ingresa esta URL en un navegador web, obtendrá la documentación de este módulo.

También tenga en cuenta que configuramos `trainable=True` al crear `hub.KerasLayer`. De esta manera, el codificador de oraciones universal previamente entrenado se ajusta durante el entrenamiento: algunos de sus pesos se ajustan mediante backprop. No todos los módulos de TensorFlow Hub se pueden ajustar con precisión, así que asegúrese de consultar la documentación de cada módulo previamente entrenado que le interese.

Después del entrenamiento, este modelo debería alcanzar una precisión de validación de más del 90 %. Eso es realmente bueno: si intentas realizar la tarea tú mismo, probablemente lo harás solo un poco mejor, ya que muchas reseñas contienen comentarios tanto positivos como negativos. Clasificar estas reseñas ambiguas es como lanzar una moneda.

Hasta ahora hemos analizado la generación de texto utilizando un char-RNN y el análisis de sentimientos con modelos RNN a nivel de palabra (basados en incrustaciones entrenables) y el uso de un potente modelo de lenguaje preentrenado de TensorFlow Hub. En la siguiente sección, exploraremos otra tarea importante de PNL: la traducción automática neuronal (NMT).


# Una red de codificador-decodificador para la traducción automática neuronal


Comencemos con un modelo simple de NMT⁠ que traducirá las oraciones en inglés al español (ver Figura 16-3).

En resumen, la arquitectura es la siguiente: las oraciones en inglés se alimentan como entradas al codificador, y el decodificador genera las traducciones al español. Tenga en cuenta que las traducciones al español también se utilizan como entradas para el decodificador durante el entrenamiento, pero se desplazan hacia atrás en un paso. En otras palabras, durante el entrenamiento, al decodificador se le da como entrada la palabra que debe tener salida en el paso anterior, independientemente de lo que realmente produzca. Esto se llama forzamiento del profesor, una técnica que acelera significativamente el entrenamiento y mejora el rendimiento del modelo. Para la primera palabra, al decodificador se le da el token de inicio de secuencia (SOS), y se espera que el decodificador termine la oración con un token de fin de secuencia (EOS).

Cada palabra está inicialmente representada por su ID (por ejemplo, `854` para la palabra "fútbol"). A continuación, una capa de `Embedding` devuelve la palabra incrustación. Estas incrustaciones de palabras luego se envían al codificador y al decodificador.

En cada paso, el decodificador genera una puntuación para cada palabra en el vocabulario de salida (es decir, español), luego la función de activación softmax convierte estas puntuaciones en probabilidades. Por ejemplo, en el primer paso la palabra "Yo" puede tener una probabilidad del 7%, "Yo" puede tener una probabilidad del 1%, y así sucesivamente. Se emite la palabra con mayor probabilidad. Esto es muy parecido a una tarea de clasificación normal y, de hecho, puedes entrenar el modelo utilizando la pérdida `"sparse_categorical_crossentropy"`, muy parecido a lo que hicimos en el modelo char-RNN.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1603.png)


(_Figura 16-3. Un modelo simple de traducción automática_)


Tenga en cuenta que en el momento de la inferencia (después del entrenamiento), no tendrá la oración objetivo para alimentar al decodificador. En su lugar, debe alimentarlo con la palabra que acaba de salir en el paso anterior, como se muestra en la Figura 16-4 (esto requerirá una búsqueda de incrustación que no se muestra en el diagrama).

#### TIP

En un documento de 2015, Samy Bengio et al. propusieron cambiar gradualmente de alimentar al decodificador el token de destino anterior a alimentarlo con el token de salida anterior durante el entrenamiento.

#### --------------------------------------------------------------------------------------

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1604.png)

(_Figura 16-4. En el momento de la inferencia, el decodificador se alimenta como entrada de la palabra que acaba de emitir en el paso de tiempo anterior_)

¡Constrúyamos y entrenemos este modelo! Primero, necesitamos descargar un conjunto de datos de pares de oraciones en inglés/español:⁠

In [ ]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",
                               extract=True)
text = (Path(path).with_name("spa-eng") / "spa.txt").read_text()

Cada línea contiene una frase en inglés y su correspondiente traducción al español, separadas por una tabulación. Comenzaremos eliminando los caracteres en español “” y “¿”, que la capa `TextVectorization` no maneja, luego analizaremos los pares de oraciones y los mezclaremos. Finalmente, los dividiremos en dos listas separadas, una por idioma:

In [ ]:
import numpy as np

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)  # separates the pairs into 2 lists

Echemos un vistazo a los tres primeros pares de frases:

In [ ]:
for i in range(3):
    print(sentences_en[i], "=>", sentences_es[i])

'''
How boring! => Qué aburrimiento!
I love sports. => Adoro el deporte.
Would you like to swap jobs? => Te gustaría que intercambiemos los trabajos?
'''

A continuación, vamos a crear dos capas `TextVectorization`, una por idioma, y adaptarlas al texto:

In [ ]:
vocab_size = 1000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_es = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

Hay algunas cosas a tener en cuenta aquí:

- Limitamos el tamaño del vocabulario a 1.000, que es bastante pequeño. Eso se debe a que el conjunto de entrenamiento no es muy grande, y porque usar un valor pequeño acelerará el entrenamiento. Los modelos de traducción de última generación suelen utilizar un vocabulario mucho más grande (por ejemplo, 30 000), un conjunto de entrenamiento mucho más grande (gigabytes) y un modelo mucho más grande (cientos o incluso miles de megabytes). Por ejemplo, echa un vistazo a los modelos Opus-MT de la Universidad de Helsinki, o al modelo M2M-100 de Facebook.

* Dado que todas las oraciones en el conjunto de datos tienen un máximo de 50 palabras, configuramos `output_sequence_length` en 50: de esta manera las secuencias de entrada se completarán automáticamente con ceros hasta que tengan 50 tokens. Si hubiera alguna oración de más de 50 fichas en el conjunto de entrenamiento, se recortaría a 50 fichas.

- Para el texto en español, añadimos "startofseq" y "endofseq" a cada oración al adaptar la capa `TextVectorization`: usaremos estas palabras como tokens SOS y EOS. Podrías usar cualquier otra palabra, siempre y cuando no sean palabras en español reales.

Inspeccionemos las primeras 10 fichas en ambos vocabularios. Comienzan con el token de relleno, el token desconocido, los tokens SOS y EOS (solo en el vocabulario español), luego las palabras reales, ordenadas por frecuencia decreciente:

In [ ]:
text_vec_layer_en.get_vocabulary()[:10]

'''
['', '[UNK]', 'the', 'i', 'to', 'you', 'tom', 'a', 'is', 'he']
'''

text_vec_layer_es.get_vocabulary()[:10]

'''
['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']
'''

A continuación, vamos a crear el conjunto de entrenamiento y el conjunto de validación (también podrías crear un conjunto de pruebas si lo necesitas). Utilizaremos los primeros 100.000 pares de oraciones para el entrenamiento y el resto para la validación. Las entradas del decodificador son las oraciones en español más un prefijo de token SOS. Los objetivos son las frases en español más un sufijo EOS:

In [ ]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

Vale, ahora estamos listos para construir nuestro modelo de traducción. Utilizaremos la API funcional para eso, ya que el modelo no es secuencial. Requiere dos entradas de texto, una para el codificador y otra para el decodificador, así que comencemos con eso:

In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

A continuación, debemos codificar estas oraciones usando las capas `TextVectorization` que preparamos anteriormente, seguidas de una capa de `Embedding` para cada idioma, con `mask_zero=True` para garantizar que el enmascaramiento se maneje automáticamente. El tamaño de incrustación es un hiperparámetro que puedes ajustar, como siempre:

In [ ]:
embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

#### TIP

Cuando los idiomas comparten muchas palabras, puede obtener un mejor rendimiento utilizando la misma capa de incrustación tanto para el codificador como para el decodificador.

#### --------------------------------------------------------------------------------------

Ahora vamos a crear el codificador y pasarle las entradas incrustadas:

In [ ]:
encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

Para simplificar las cosas, solo usamos una única capa `LSTM`, pero puedes apilar varias. También configuramos `return_state=True` para obtener una referencia al estado final de la capa. Como usamos una capa `LSTM`, en realidad hay dos estados: el estado a corto plazo y el estado a largo plazo. La capa devuelve estos estados por separado, por lo que tuvimos que escribir `*encoder_state` para agrupar ambos estados en una lista.⁠ Ahora podemos usar este (doble) estado como estado inicial del decodificador:

In [ ]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

A continuación, podemos pasar las salidas del decodificador a través de una capa `Dense` con la función de activación softmax para obtener las probabilidades de palabras para cada paso:

In [ ]:
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

#### OPTIMIZACIÓN DE LA CAPA DE SALIDA

Cuando el vocabulario de salida es grande, generar una probabilidad para todas y cada una de las palabras posibles puede ser bastante lento. Si el vocabulario de destino contuviera, digamos, 50.000 palabras en español en lugar de 1.000, entonces el decodificador generaría vectores de 50.000 dimensiones, y calcular la función softmax sobre un vector tan grande sería muy intensivo desde el punto de vista computacional. Para evitar esto, una solución es observar únicamente los logits generados por el modelo para la palabra correcta y una muestra aleatoria de palabras incorrectas, y luego calcular una aproximación de la pérdida basada únicamente en estos logits. Esta técnica softmax muestreada fue introducida en 2015 por Sébastien Jean et al.⁠ En TensorFlow puede usar la función `tf.nn.sampled_softmax_loss()` para esto durante el entrenamiento y usar la función softmax normal en el momento de la inferencia (softmax muestreado no se puede usar en tiempo de inferencia porque requiere conocer el objetivo).

Otra cosa que puedes hacer para acelerar el entrenamiento, que es compatible con el softmax muestreado, es vincular los pesos de la capa de salida a la transposición de la matriz de incrustación del decodificador (verás cómo atar los pesos en el capítulo 17). Esto reduce significativamente el número de parámetros del modelo, lo que acelera el entrenamiento y, a veces, también puede mejorar la precisión del modelo, especialmente si no tiene muchos datos de entrenamiento. La matriz de incrustación es equivalente a la codificación de un solo calor seguida de una capa lineal sin término de sesgo y sin función de activación que asigna los vectores de un solo calor al espacio de incrustación. La capa de salida hace lo contrario. Por lo tanto, si el modelo puede encontrar una matriz de incrustación cuya transposición esté cerca de su inversa (tal matriz se llama matriz ortogonal), entonces no hay necesidad de aprender un conjunto separado de pesos para la capa de salida.

#### ----------------------------------------------------------------------------------

Y eso es todo! Solo necesitamos crear el `Model` Keras, compilarlo y entrenarlo:

In [ ]:
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Después del entrenamiento, podemos usar el modelo para traducir nuevas oraciones en inglés al español. Pero no es tan simple como llamar a model.predict(), porque el decodificador espera como entrada la palabra que se predijo en el paso de tiempo anterior. Una forma de hacer esto es escribir una celda de memoria personalizada que haga un seguimiento de la salida anterior y la alimente al codificador en el siguiente paso. Sin embargo, para simplificar las cosas, podemos llamar al modelo varias veces, prediciendo una palabra extra en cada ronda. Escribamos una pequeña función de utilidad para eso:

In [ ]:
def translate(sentence_en):
    translation = ""
    for word_idx in range(max_length):
        X = np.array([sentence_en])  # encoder input
        X_dec = np.array(["startofseq " + translation])  # decoder input
        y_proba = model.predict((X, X_dec))[0, word_idx]  # last token's probas
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
    return translation.strip()

La función simplemente sigue prediciendo una palabra a la vez, completando gradualmente la traducción, y se detiene una vez que llega al token EOS. ¡Vamos a intentarlo!

In [ ]:
translate("I like soccer")

'''
'me gusta el fútbol'
'''

¡Hurra, funciona! Bueno, al menos lo hace con frases muy cortas. Si intentas jugar con este modelo por un tiempo, encontrarás que aún no es bilingüe y, en particular, realmente tiene problemas con frases más largas. Por ejemplo:

In [ ]:
translate("I like soccer and also going to the beach")

'''
'me gusta el fútbol y a veces mismo al bus'
'''

La traducción dice “Me gusta el fútbol y a veces hasta el autobús”. Entonces, ¿cómo puedes mejorarlo? Una forma es aumentar el tamaño del conjunto de entrenamiento y agregar más capas LSTM tanto en el codificador como en el decodificador. Pero esto sólo le llevará hasta cierto punto, así que veamos técnicas más sofisticadas, comenzando con capas recurrentes bidireccionales.


## RNN bidireccionales


En cada paso de tiempo, una capa recurrente regular solo observa las entradas pasadas y presentes antes de generar su salida. En otras palabras, es causal, lo que significa que no puede mirar hacia el futuro. Este tipo de RNN tiene sentido al pronosticar series temporales, o en el decodificador de un modelo de secuencia a secuencia (seq2seq). Pero para tareas como la clasificación de texto, o en el codificador de un modelo seq2seq, a menudo es preferible mirar hacia adelante en las siguientes palabras antes de codificar una palabra determinada.

Por ejemplo, considere las frases "el brazo derecho", "la persona adecuada" y "el derecho a criticar": para codificar adecuadamente la palabra "correcto", debe mirar hacia adelante. Una solución es ejecutar dos capas recurrentes en las mismas entradas, una leyendo las palabras de izquierda a derecha y la otra leyéndolas de derecha a izquierda, y luego combinar sus salidas en cada paso de tiempo, generalmente concatenandolas. Esto es lo que hace una capa recurrente bidireccional (ver Figura 16-5).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1605.png)

(_Figura 16-5. Una capa recurrente bidireccional_)

Para implementar una capa recurrente bidireccional en Keras, simplemente envuelva una capa recurrente en una capa `tf.keras.layers.Bidireccional`. Por ejemplo, la siguiente capa bidireccional podría usarse como codificador en nuestro modelo de traducción:

In [ ]:
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_state=True))

#### NOTA

La capa `Bidirectional` creará un clon de la capa `GRU` (pero en la dirección inversa), ejecutará ambas y concatenará sus salidas. Entonces, aunque la capa `GRU` tiene 10 unidades, la capa `Bidirectional` generará 20 valores por paso de tiempo.

#### ---------------------------------------------- ----------------------------------------

Sólo hay un problema. Esta capa ahora devolverá cuatro estados en lugar de dos: los estados finales a corto y largo plazo de la capa `LSTM` directa, y los estados finales a corto y largo plazo de la capa `LSTM` inversa. No podemos usar este estado cuádruple directamente como el estado inicial de la capa `LSTM` del decodificador, ya que solo espera dos estados (a corto y largo plazo). No podemos hacer que el decodificador sea bidireccional, ya que debe seguir siendo causal: de lo contrario, haría trampa durante el entrenamiento y no funcionaría. En cambio, podemos concatenar los dos estados de corto plazo y también concatenar los dos estados de largo plazo:

In [ ]:
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # short-term (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # long-term (1 & 3)

Ahora echemos un vistazo a otra técnica popular que puede mejorar en gran medida el rendimiento de un modelo de traducción en el momento de la inferencia: la búsqueda de haces.


## Búsqueda de vigas

Supongamos que has entrenado un modelo de codificador-decodificador y lo usas para traducir la frase "Me gusta el fútbol" al español. Esperas que produzca la traducción adecuada "me gusta el fútbol", pero desafortunadamente produce "me gustan los jugadores", que significa "me gustan los jugadores". Mirando el conjunto de entrenamiento, notas muchas frases como "Me gustan los coches", que se traduce como "me gustan los autos", por lo que no fue absurdo que el modelo emitiera "me gustan los" después de ver "me gusta". Desafortunadamente, en este caso fue un error, ya que el "fértol" es singular. El modelo no pudo volver atrás y arreglarlo, por lo que trató de completar la oración lo mejor que pudo, en este caso usando la palabra "jugadores". ¿Cómo podemos darle al modelo la oportunidad de volver atrás y corregir los errores que cometió antes? Una de las soluciones más comunes es la búsqueda de haces: realiza un seguimiento de una breve lista de las k oraciones más prometedoras (por ejemplo, las tres primeras), y en cada paso del decodificador intenta extenderlas en una palabra, manteniendo solo las k oraciones más probables. El parámetro k se llama ancho de haz.

Por ejemplo, supongamos que usas el modelo para traducir la frase "Me gusta el fútbol" usando la búsqueda de haz con un ancho de haz de 3 (ver Figura 166). En el primer paso del decodificador, el modelo producirá una probabilidad estimada para cada primera palabra posible en la oración traducida. Supongamos que las tres palabras principales son "yo" (75% de probabilidad estimada), "a" (3%) y "como" (1%). Esa es nuestra lista corta hasta ahora. A continuación, usamos el modelo para encontrar la siguiente palabra para cada frase. Para la primera frase ("yo"), tal vez el modelo genere una probabilidad del 36 % para la palabra "gustan", del 32 % para la palabra "gusta", el 16 % para la palabra "encanta", y así suceder. Tenga en cuenta que estas son en realidad probabilidades condicionales, dado que la oración comienza con "yo". Para la segunda oración ("a"), el modelo podría producir una probabilidad condicional del 50 % para la palabra "mi", y así sucede. Suponiendo que el vocabulario tenga 1000 palabras, terminaremos con 1000 probabilidades por oración.

A continuación, calculamos las probabilidades de cada una de las 3.000 oraciones de dos palabras que consideramos (3 × 1.000). Hacemos esto multiplicando la probabilidad condicional estimada de cada palabra por la probabilidad estimada de la oración que completa. Por ejemplo, la probabilidad estimada de la oración "yo" fue del 75 %, mientras que la probabilidad condicional estimada de la palabra "gustan" (dado que la primera palabra es "yo") fue del 36 %, por lo que la probabilidad estimada de la oración "me gustan" es del 75 % × 36 % = 27 %. Después de calcular las probabilidades de las 3.000 oraciones de dos palabras, mantenemos solo las 3 primeras. En este ejemplo, todos comienzan con la palabra "me": "me gustan" (27%), "me gusta" (24%) y "me encanta" (12 %). En este momento, la frase "me gustan" está ganando, pero "me gusta" no ha sido eliminada.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1606.png)

(_Figura 16-6. Búsqueda de vigas, con un ancho de viga de 3_)


Luego repetimos el mismo proceso: usamos el modelo para predecir la siguiente palabra en cada una de estas tres oraciones, y calculamos las probabilidades de las 3.000 oraciones de tres palabras que consideramos. Tal vez los tres primeros sean ahora "me gustan los" (10%), "me gusta el" (8%) y "me gusta mucho" (2%). En el siguiente paso podemos conseguir "me gusta el fútbol" (6%), "me gusta mucho el" (1%) y "me gusta el deporte" (0,2 %). Tenga en cuenta que "me gustan" fue eliminado, y la traducción correcta ya está por delante. Aumentamos el rendimiento de nuestro modelo de codificador-decodificador sin ningún entrenamiento adicional, simplemente usándolo de manera más inteligente.

#### TIP

La biblioteca TensorFlow Addons incluye una API completa de seq2seq que le permite crear modelos de codificador-decodificador con atención, incluida la búsqueda de haces y más. Sin embargo, su documentación es actualmente muy limitada. Implementar la búsqueda de vigas es un buen ejercicio, ¡así que pruébalo! Echa un vistazo al cuaderno de este capítulo para encontrar una posible solución.

#### -------------------------------------------------------------------------------------

Con todo esto, puedes obtener traducciones razonablemente buenas para oraciones bastante cortas. Desafortunadamente, este modelo será muy malo para traducir oraciones largas. Una vez más, el problema proviene de la limitada memoria a corto plazo de los RNN. Los mecanismos de atención son la innovación que cambió las reglas del juego que abordó este problema.


# Mecanismos de atención

Considere el camino desde la palabra "fútbol" hasta su traducción "fútbol" en la Figura 16-3: ¡es bastante largo! Esto significa que una representación de esta palabra (junto con todas las demás palabras) debe llevarse a cabo en muchos pasos antes de que se use realmente. ¿No podemos hacer este camino más corto?

Esta fue la idea central en un documento histórico de 2014⁠ de Dzmitry Bahdanau et al., donde los autores introdujeron una técnica que permitía al decodificador centrarse en las palabras apropiadas (codificadas por el codificador) en cada paso del tiempo. Por ejemplo, en el momento en el que el decodificador necesita emitir la palabra "fútbol", centrará su atención en la palabra "fútbol". Esto significa que el camino desde una palabra de entrada hasta su traducción es ahora mucho más corto, por lo que las limitaciones de memoria a corto plazo de los RNN tienen mucho menos impacto. Los mecanismos de atención revolucionaron la traducción automática neuronal (y el aprendizaje profundo en general), lo que permitió una mejora significativa en el estado del arte, especialmente para oraciones largas (por ejemplo, más de 30 palabras).

#### NOTA

La métrica más común utilizada en NMT es la puntuación del subestudio de evaluación bilingüe (BLEU), que compara cada traducción producida por el modelo con varias traducciones buenas producidas por humanos: cuenta el número de n-gramos (secuencias de n palabras) que aparecen en cualquiera de las traducciones objetivo y ajusta la puntuación para tener en cuenta la frecuencia de los n-gramos producidos en las traducciones objetivo.

#### -------------------------------------------------------------------------------------

La figura 16-7 muestra nuestro modelo de codificador-decodificador con un mecanismo de atención adicional. A la izquierda, tienes el codificador y el decodificador. En lugar de simplemente enviar el estado oculto final del codificador al decodificador, así como la palabra de destino anterior en cada paso (que todavía se hace, aunque no se muestra en la figura), ahora también enviamos todas las salidas del codificador al decodificador. Dado que el decodificador no puede lidiar con todas estas salidas del codificador a la vez, deben agregarse: en cada paso de tiempo, la celda de memoria del decodificador calcula una suma ponderada de todas las salidas del codificador. Esto determina en qué palabras se centrará en este paso. El peso α(t,i) es el peso de la salida del ensésima salida del codificador en el tésimo paso de tiempo del decodificador. Por ejemplo, si el peso α(3,2) es mucho mayor que los pesos α(3,0) y α(3,1), entonces el decodificador prestará mucha más atención a la salida del codificador para la palabra #2 ("futásto") que a las otras dos salidas, al menos en este paso. El resto del decodificador funciona igual que antes: en cada paso de tiempo, la celda de memoria recibe las entradas que acabamos de discutir, además del estado oculto del paso de tiempo anterior, y finalmente (aunque no está representado en el diagrama) recibe la palabra de destino del paso de tiempo anterior (o en el momento de inferencia, la salida del paso de tiempo anterior).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1607.png)

(_Figura 16-7. Traducción automática neuronal utilizando una red de codificador-decodificador con un modelo de atención_)

¿Pero de dónde vienen estos pesos α(t,i)? Bueno, son generados por una pequeña red neuronal llamada modelo de alineación (o capa de atención), que se entrena conjuntamente con el resto del modelo codificador-decodificador. Este modelo de alineación se ilustra en el lado derecho de la Figura 16-7. Comienza con una capa Densa compuesta por una sola neurona que procesa cada una de las salidas del codificador, junto con el estado oculto anterior del decodificador (por ejemplo, h(2)). Esta capa genera una puntuación (o energía) para cada salida del codificador (por ejemplo, e(3, 2)): esta puntuación mide qué tan bien está alineada cada salida con el estado oculto anterior del decodificador. Por ejemplo, en la Figura 16-7, el modelo ya generó “me gusta el”, por lo que ahora espera un sustantivo: la palabra “soccer” es la que mejor se alinea con el estado actual. por lo que obtiene una puntuación alta. Finalmente, todas las puntuaciones pasan por una capa softmax para obtener un peso final para cada salida del codificador (por ejemplo, α(3,2)). Todos los pesos para un paso de tiempo dado del decodificador suman 1. Este mecanismo de atención particular se llama atención de Bahdanau (llamado así por el primer autor del artículo de 2014). Dado que concatena la salida del codificador con el estado oculto anterior del decodificador, a veces se le llama atención concatenativa (o atención aditiva).

#### NOTA

Si la oración de entrada tiene n palabras de largo, y suponiendo que la oración de salida sea aproximadamente la misma larga, entonces este modelo tendrá que calcular alrededor de n2 pesos. Afortunadamente, esta complejidad computacional cuadrática sigue siendo manejable porque incluso las oraciones largas no tienen miles de palabras.

#### --------------------------------------------------------------------------------------

Otro mecanismo de atención común, conocido como atención de Luong o atención multiplicativa, fue propuesto poco después, en 2015, por Minh-Thang Luong et al. Debido a que el objetivo del modelo de alineación es medir la similitud entre una de las salidas del codificador y el estado oculto anterior del decodificador, los autores propusieron simplemente calcular el producto de puntos (ver Capítulo 4) de estos dos vectores, ya que esta es a menudo una medida de similitud bastante buena, y el hardware moderno puede calcularlo de manera muy eficiente. Para que esto sea posible, ambos vectores deben tener la misma dimensionalidad. El producto de puntos da una puntuación, y todas las puntuaciones (en un paso de tiempo de decodificador dado) pasan por una capa softmax para dar los pesos finales, al igual que en la atención de Bahdanau. Otra simplificación propuesta por Luong et al. fue usar el estado oculto del decodificador en el paso de tiempo actual en lugar de en el paso de tiempo anterior (es decir, h(t) en lugar de h(t-1)), y luego usar la salida del mecanismo de atención (notado
h~(T)) directamente para calcular las predicciones del decodificador, en lugar de usarlo para calcular el estado oculto actual del decodificador. Los investigadores también propusieron una variante del mecanismo del producto de dot en el que las salidas del codificador pasan primero a través de una capa completamente conectada (sin un término de sesgo) antes de que se calculen los productos de dot. Esto se llama el enfoque de producto de punto "general". Los investigadores compararon ambos enfoques de productos de puntos con el mecanismo de atención concatenativa (agregando un vector de parámetros de reescalado v), y observaron que las variantes de productos de puntos se desempeñaron mejor que la atención concatenativa. Por esta razón, la atención concatenativa se usa mucho menos ahora. Las ecuaciones para estos tres mecanismos de atención se resumen en la ecuación 16-1.


### Ecuación 16-1. Mecanismos de atención

<a href="https://ibb.co/ZTLBnnh"><img src="https://i.ibb.co/q1yNttR/Captura-de-pantalla-2024-03-25-a-las-2-19-44.png" alt="Captura-de-pantalla-2024-03-25-a-las-2-19-44" border="0"></a>

Keras proporciona una capa `tf.keras.layers.Attention` para la atención de Luong y una capa AdditiveAttention para la atención de Bahdanau. Agreguemos la atención de Luong a nuestro modelo codificador-decodificador. Dado que necesitaremos pasar todas las salidas del codificador a la capa de `Atention`, primero debemos configurar `return_sequences=True` al crear el codificador:

In [ ]:
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_sequences=True, return_state=True))

A continuación, tenemos que crear la capa de atención y pasarla por los estados del decodificador y las salidas del codificador. Sin embargo, para acceder a los estados del decodificador en cada paso, tendríamos que escribir una celda de memoria personalizada. Para simplificar, usemos las salidas del decodificador en lugar de sus estados: en la práctica, esto también funciona bien, y es mucho más fácil de codificar. Luego simplemente pasamos las salidas de la capa de atención directamente a la capa de salida, como se sugiere en el documento de atención de Luong:

In [ ]:
attention_layer = tf.keras.layers.Attention()
attention_outputs = attention_layer([decoder_outputs, encoder_outputs])
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(attention_outputs)

¡Y eso es todo! Si entrenas este modelo, encontrarás que ahora maneja oraciones mucho más largas. Por ejemplo:

In [ ]:
translate("I like soccer and also going to the beach")

'''
'me gusta el fútbol y también ir a la playa'
'''

En resumen, la capa de atención proporciona una forma de centrar la atención del modelo en parte de las entradas. Pero hay otra forma de pensar en esta capa: actúa como un mecanismo de recuperación de memoria diferenciable.

Por ejemplo, supongamos que el codificador analizó la oración de entrada "Me gusta el fútbol", y se las arregló para entender que la palabra "I" es el sujeto y la palabra "me gusta" es el verbo, por lo que codificó esta información en sus salidas para estas palabras. Ahora supongamos que el decodificador ya ha traducido el tema, y piensa que debería traducir el verbo a continuación. Para ello, necesita obtener el verbo de la oración de entrada. Esto es análogo a una búsqueda en el diccionario: es como si el codificador hubiera creado un diccionario {"sujeto": "Ellos", "verbo": "jugado", ... } y el decodificador quisiera buscar el valor que corresponde a la clave "verbo".

Sin embargo, el modelo no tiene tokens discretos para representar las claves (como "sujeto" o "verbo"); en su lugar, tiene representaciones vectorizadas de estos conceptos que aprendió durante el entrenamiento, por lo que la consulta que utilizará para la búsqueda no coincidirá perfectamente con ninguna clave en el diccionario. La solución es calcular una medida de similitud entre la consulta y cada clave del diccionario, y luego usar la función softmax para convertir estas puntuaciones de similitud en pesos que suman 1. Como vimos antes, eso es exactamente lo que hace la capa de atención. Si la clave que representa el verbo es, con mucho, la más similar a la consulta, entonces el peso de esa clave estará cerca de 1.

A continuación, la capa de atención calcula una suma ponderada de los valores correspondientes: si el peso de la clave "verbo" está cerca de 1, entonces la suma ponderada estará muy cerca de la representación de la palabra "jugada".

Es por eso que las capas Keras Attention y AdditiveAttention esperan una lista como entrada, que contiene dos o tres elementos: las consultas, las claves y, opcionalmente, los valores. Si no pasa ningún valor, automáticamente serán iguales a las claves. Entonces, mirando nuevamente el ejemplo de código anterior, las salidas del decodificador son las consultas y las salidas del codificador son tanto las claves como los valores. Para cada salida del decodificador (es decir, cada consulta), la capa de atención devuelve una suma ponderada de las salidas del codificador (es decir, las claves/valores) que son más similares a la salida del decodificador.

La conclusión es que un mecanismo de atención es un sistema de recuperación de memoria que se puede entrenar. Es tan potente que en realidad puedes construir modelos de última generación utilizando solo mecanismos de atención. Introduzca la arquitectura del transformador.


## La atención es todo lo que necesitas (attention is all what you need): la arquitectura original del transformador


En un innovador documento de 2017, un equipo de investigadores de Google sugirió que "la atención es todo lo que necesitas". Crearon una arquitectura llamada transformador, que mejoró significativamente el estado de la arte en NMT sin usar ninguna capa recurrente o convolucional, ⁠21 solo mecanismos de atención (además de capas de incrustación, capas densas, capas de normalización y algunos otros fragmentos). Debido a que el modelo no es recurrente, no sufre tanto de los problemas de gradientes de desaparición o explosión como los RNN, se puede entrenar en menos pasos, es más fácil de paralelizar a través de múltiples GPU y puede capturar mejor los patrones de largo alcance que los RNN. La arquitectura original del transformador de 2017 está representada en la Figura 16-8.

En resumen, la parte izquierda de la Figura 16-8 es el codificador, y la parte derecha es el decodificador. Cada capa de incrustación genera un tensor 3D de forma [tamaño del lote, longitud de la secuencia, tamaño de incrustación]. Después de eso, los tensores se transforman gradualmente a medida que fluyen a través del transformador, pero su forma sigue siendo la misma.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1608.png)

(_Figura 16-8. La arquitectura original del transformador de 2017⁠_)

Si utiliza el transformador para NMT, durante el entrenamiento debe alimentar las oraciones en inglés al codificador y las traducciones al español correspondientes al decodificador, con un token SOS adicional insertado al comienzo de cada oración. En el momento de la inferencia, debe llamar al transformador varias veces, produciendo las traducciones una palabra a la vez y alimentando las traducciones parciales al decodificador en cada ronda, al igual que hicimos anteriormente en la función `translate()`.

El papel del codificador es transformar gradualmente las entradas (representaciones de palabras de la oración en inglés) hasta que la representación de cada palabra capture perfectamente el significado de la palabra, en el contexto de la oración. Por ejemplo, si alimentas al codificador con la frase "Me gusta el fútbol", entonces la palabra "me gusta" comenzará con una representación bastante vaga, ya que esta palabra podría significar cosas diferentes en diferentes contextos: piensa en "Me gusta el fútbol" frente a "Es así". Pero después de pasar por el codificador, la representación de la palabra debe capturar el significado correcto de "me gusta" en la oración dada (es decir, para gustar), así como cualquier otra información que pueda ser necesaria para la traducción (por ejemplo, es un verbo).

El papel del decodificador es transformar gradualmente cada representación de palabras en la oración traducida en una representación de palabras de la siguiente palabra en la traducción. Por ejemplo, si la oración a traducir es "Me gusta el fútbol", y la oración de entrada del decodificador es "<SOS> me gusta el fútbol", entonces después de pasar por el decodificador, la representación de la palabra "el" terminará transformada en una representación de la palabra "fútbol". Del mismo modo, la representación de la palabra "fútbol" se transformará en una representación del token EOS.
    
Después de pasar por el decodificador, cada representación de palabra pasa por una capa Densa final con una función de activación softmax, que con suerte generará una alta probabilidad para la siguiente palabra correcta y una baja probabilidad para todas las demás palabras. La frase prevista debería ser “me gusta el fútbol <EOS>”.
    
    
Ese fue el panorama general; ahora repasemos la Figura 16-8 con más detalle:

* En primer lugar, tenga en cuenta que tanto el codificador como el decodificador contienen módulos que se apilan N veces. En el artículo, N = 6. Las salidas finales de toda la pila de codificadores se alimentan al decodificador en cada uno de estos N niveles.

- Al hacer zoom, puede ver que ya está familiarizado con la mayoría de los componentes: hay dos capas de incrustación; varias conexiones de salto, cada una de ellas seguida de una capa de normalización de capa; varios módulos de avance que se componen de dos capas densas cada una (la primera que utiliza la función de activación ReLU, la segunda sin función de activación); y, finalmente, la capa de salida es una capa densa que utiliza la función de activación softmax. También puede rociar un poco de abandono después de las capas de atención y los módulos de alimentación, si es necesario. Dado que todas estas capas se distribuyen en el tiempo, cada palabra se trata de forma independiente de todas las demás. Pero, ¿cómo podemos traducir una oración mirando las palabras completamente por separado? Bueno, no podemos, así que ahí es donde entran en adí los nuevos componentes:
La capa de atención de varias cabezas del codificador actualiza cada representación de palabras atendiendo a (es decir, prestando atención a) todas las demás palabras de la misma oración. Ahí es donde la vaga representación de la palabra "me gusta" se convierte en una representación más rica y precisa, capturando su significado preciso en la oración dada. En breve discutiremos exactamente cómo funciona esto.

    * La capa de atención multicabeza enmascarada del decodificador hace lo mismo, pero cuando procesa una palabra, no atiende a las palabras que se encuentran después de ella: es una capa causal. Por ejemplo, cuando procesa la palabra "gusta", solo atiende a las palabras "<SOS> me gusta", e ignora las palabras "el fútbol" (o de lo contrario eso sería hacer trampa).

    - La capa superior de atención multicabeza del decodificador es donde el decodificador presta atención a las palabras de la oración en inglés. Esto se llama atención cruzada, no atención personal en este caso. Por ejemplo, el decodificador probablemente prestará mucha atención a la palabra "fútbol" cuando procese la palabra "el" y transforme su representación en una representación de la palabra "fútbol".

    * Las codificaciones posicionales son vectores densos (al igual que las incrustaciones de palabras) que representan la posición de cada palabra en la oración. La enésima codificación posicional se añade a la incrustación de la palabra enésima palabra en cada oración. Esto es necesario porque todas las capas de la arquitectura del transformador ignoran las posiciones de las palabras: sin codificaciones posicionales, se podrían barajar las secuencias de entrada, y simplemente barajaría las secuencias de salida de la misma manera. Obviamente, el orden de las palabras importa, por lo que necesitamos dar información posicional al transformador de alguna manera: agregar codificaciones posicionales a las representaciones de palabras es una buena manera de lograrlo.

#### NOTA

Las dos primeras flechas que van a cada capa de atención multicabeza en la Figura 16-8 representan las claves y los valores, y la tercera flecha representa las consultas. En las capas de autoatención, las tres son iguales a las representaciones de palabras que emite la capa anterior, mientras que en la capa de atención superior del decodificador, las teclas y los valores son iguales a las representaciones de palabras finales del codificador, y las consultas son iguales a las representaciones de palabras que emite la capa anterior.

#### --------------------------------------------------------------------------------------

Revisemos los nuevos componentes de la arquitectura del transformador con más detalle, comenzando con las codificaciones posicionales.


## Codificaciones posicionales


Una codificación posicional es un vector denso que codifica la posición de una palabra dentro de una oración: la iésima codificación posicional se agrega al `Embedding` de la iésima palabra en la oración. La forma más sencilla de implementar esto es usar una capa de incrustación y hacer que codifique todas las posiciones desde 0 hasta la longitud máxima de secuencia en el lote, luego agregar el resultado a las incrustaciones de palabras. Las reglas de transmisión garantizarán que las codificaciones posicionales se apliquen a cada secuencia de entrada. Por ejemplo, aquí se explica cómo agregar codificaciones posicionales a las entradas del codificador y del decodificador:

In [ ]:
max_length = 50  # max length in the whole training set
embed_size = 128
pos_embed_layer = tf.keras.layers.Embedding(max_length, embed_size)
batch_max_len_enc = tf.shape(encoder_embeddings)[1]
encoder_in = encoder_embeddings + pos_embed_layer(tf.range(batch_max_len_enc))
batch_max_len_dec = tf.shape(decoder_embeddings)[1]
decoder_in = decoder_embeddings + pos_embed_layer(tf.range(batch_max_len_dec))

Tenga en cuenta que esta implementación supone que las incrustaciones se representan como tensores regulares, no como tensores irregulares. El codificador y el decodificador comparten la misma capa de `Embedding` para las codificaciones posicionales, ya que tienen el mismo tamaño de incrustación (este suele ser el caso).

En lugar de usar codificaciones posicionales entrenables, los autores del documento del transformador eligieron usar codificaciones posicionales fijas, basadas en las funciones del seno y el coseno en diferentes frecuencias. La matriz de codificación posicional P se define en la ecuación 16-2 y se representa en la parte superior de la Figura 16-9 (transpuesta), donde Pp,i es el i-écimo componente de la codificación de la palabra ubicada en la posición pth de la oración.


### Ecuación 16-2. Codificaciones posicionales sineno/coseno

<a href="https://imgbb.com/"><img src="https://i.ibb.co/XXRLhW5/Captura-de-pantalla-2024-03-25-a-las-2-47-13.png" alt="Captura-de-pantalla-2024-03-25-a-las-2-47-13" border="0"></a>

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1609.png)

(_Figura 16-9. Matriz de codificación posicional sineno/coseno (transpuesta, superior) con un enfoque en dos valores de i (abajo)_)

Esta solución puede dar el mismo rendimiento que las codificaciones posicionales entrenables, y puede extenderse a oraciones arbitrariamente largas sin agregar ningún parámetro al modelo (sin embargo, cuando hay una gran cantidad de datos previos al entrenamiento, generalmente se prefieren las codificaciones posicionales entrenables). Después de agregar estas codificaciones posicionales a las incrustaciones de palabras, el resto del modelo tiene acceso a la posición absoluta de cada palabra en la oración porque hay una codificación posicional única para cada posición (por ejemplo, la codificación posicional para la palabra ubicada en la posición 22 en una oración está representada por la línea con dises el diséis vertical en la parte superior izquierda de la Figura 16-9, y se puede ver que es única para esa posición). Además, la elección de las funciones oscilantes (seno y coseno) hace posible que el modelo también aprenda posiciones relativas. Por ejemplo, las palabras ubicadas con 38 palabras de distancia (por ejemplo, en las posiciones p = 22 y p = 60) siempre tienen los mismos valores de codificación posicional en las dimensiones de codificación i = 100 e i = 101, como se puede ver en la Figura 16-9. Esto explica por qué necesitamos tanto el seno como el coseno para cada frecuencia: si solo usáramos el seno (la onda azul ati = 100), el modelo no sería capaz de distinguir las posiciones p = 22 y p = 35 (marcada por una cruz).

No existe una capa `PositionalEncoding` en TensorFlow, pero no es demasiado difícil crear una. Por razones de eficiencia, precalculamos la matriz de codificación posicional en el constructor. El método `call()` simplemente trunca esta matriz de codificación a la longitud máxima de las secuencias de entrada y las agrega a las entradas. También configuramos `support_masking=True` para propagar la máscara automática de entrada a la siguiente capa:

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_length, embed_size, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        assert embed_size % 2 == 0, "embed_size must be even"
        p, i = np.meshgrid(np.arange(max_length),
                           2 * np.arange(embed_size // 2))
        pos_emb = np.empty((1, max_length, embed_size))
        pos_emb[0, :, ::2] = np.sin(p / 10_000 ** (i / embed_size)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10_000 ** (i / embed_size)).T
        self.pos_encodings = tf.constant(pos_emb.astype(self.dtype))
        self.supports_masking = True

    def call(self, inputs):
        batch_max_length = tf.shape(inputs)[1]
        return inputs + self.pos_encodings[:, :batch_max_length]

Usemos esta capa para añadir la codificación posicional a las entradas del codificador:

In [ ]:
pos_embed_layer = PositionalEncoding(max_length, embed_size)
encoder_in = pos_embed_layer(encoder_embeddings)
decoder_in = pos_embed_layer(decoder_embeddings)

Ahora echemos un vistazo más profundo al corazón del modelo de transformador, en la capa de atención de varias cabezas.

## Atención multicabeza (multihead)

Para entender cómo funciona una capa de atención de varias cabezas, primero debemos entender la capa de atención de producto de puntos a escala, en la que se basa. Su ecuación se muestra en la ecuación 16-3, en forma vectorial. Es lo mismo que la atención de Luong, excepto por un factor de escala.

### Ecuación 16-3. Atención a escala del producto de los dots

<a href="https://imgbb.com/"><img src="https://i.ibb.co/tLBqqvZ/Captura-de-pantalla-2024-03-25-a-las-2-53-15.png" alt="Captura-de-pantalla-2024-03-25-a-las-2-53-15" border="0"></a>

En esta ecuación:

- **Q** es una matriz que contiene una fila por consulta. Su forma es [nqueries,dkeys], donde nqueries es el número de consultas y dkeys es el número de dimensiones de cada consulta y cada clave.

* **K** es una matriz que contiene una fila por clave. Su forma es [nkeys, dkeys], donde nkeys es el número de claves y valores.

- **V** es una matriz que contiene una fila por valor. Su forma es [nkeys,dvalues], donde dvalues es el número de dimensiones de cada valor.

* La forma de Q K⊺ es [nqueries, nkeys]: contiene una puntuación de similitud para cada par de consultas/claves. Para evitar que esta matriz sea enorme, las secuencias de entrada no deben ser demasiado largas (discutiremos cómo superar esta limitación más adelante en este capítulo). La salida de la función softmax tiene la misma forma, pero todas las filas suman 1. El resultado final tiene una forma de [consultas, dvalues]: hay una fila por consulta, donde cada fila representa el resultado de la consulta (una suma ponderada de los valores).

- El factor de escala 1/sqrt(Dkeys) reduce las puntuaciones de similitud para evitar saturar la función softmax, lo que daría lugar a pequeños gradientes.

* Es posible enmascarar algunos pares clave/valor añadiendo un valor negativo muy grande a las puntuaciones de similitud correspondientes, justo antes de calcular el softmax. Esto es útil en la capa de atención de varias cabezas enmascarada.

Si configura `use_scale=True` al crear una capa `tf.keras.layers.Attention`, se creará un parámetro adicional que permitirá a la capa aprender cómo reducir correctamente las puntuaciones de similitud. La atención del producto escalado utilizada en el modelo del transformador es casi la misma, excepto que siempre escala las puntuaciones de similitud por el mismo factor, 1/sqrt(Dkeys).

Tenga en cuenta que las entradas de la capa `Attention` son como **Q, K y V**, excepto con una dimensión de lote adicional (la primera dimensión). Internamente, la capa calcula todas las puntuaciones de atención para todas las oraciones del lote con una sola llamada a `tf.matmul(queries, keys):` esto la hace extremadamente eficiente. De hecho, en TensorFlow, si A y B son tensores con más de dos dimensiones, digamos, de forma [2, 3, 4, 5] y [2, 3, 5, 6], respectivamente, entonces tf.matmul(A, B) tratará estos tensores como matrices de 2 × 3 donde cada celda contiene una matriz, y multiplicará las matrices correspondientes: la matriz en la i-ésima fila y j-ésima columna en A se multiplicará por la matriz en la i-ésima fila y j-ésima columna en B. Dado que el producto de una matriz de 4 × 5 con una matriz de 5 × 6 es una matriz de 4 × 6, `tf.matmul(A, B)` devolverá una matriz de forma [2, 3, 4, 6].

Ahora estamos listos para ver la capa de atención de varias cabezas. Su arquitectura se muestra en la Figura 16-10.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1610.png)

(_Figura 16-10. Arquitectura de capa de atención multicabeza⁠_)

Como puede ver, es solo un grupo de capas de atención de productos de puntos a escala, cada una precedida por una transformación lineal de los valores, claves y consultas (es decir, una capa densa distribuida en el tiempo sin función de activación). Todas las salidas están simplemente concatenadas, y pasan por una transformación lineal final (de nuevo, distribuida en el tiempo).

¿Pero por qué? ¿Cuál es la intuición detrás de esta arquitectura? Bueno, considera una vez más la palabra "me gusta" en la frase "Me gusta el fútbol". El codificador era lo suficientemente inteligente como para codificar el hecho de que es un verbo. Pero la representación de la palabra también incluye su posición en el texto, gracias a las codificaciones posicionales, y probablemente incluye muchas otras características que son útiles para su traducción, como el hecho de que está en tiempo presente. En resumen, la representación de la palabra codifica muchas características diferentes de la palabra. Si solo usáramos una sola capa de atención de producto de dos a escala, solo podríamos consultar todas estas características de una sola vez.

Esta es la razón por la que la capa de atención de múltiples cabezas aplica múltiples transformaciones lineales diferentes de los valores, claves y consultas: esto permite al modelo aplicar muchas proyecciones diferentes de la representación de la palabra en diferentes subespacios, cada una centrándose en un subconjunto de las características de la palabra. Tal vez una de las capas lineales proyectará la representación de la palabra en un subespacio donde todo lo que queda es la información de que la palabra es un verbo, otra capa lineal extraerá solo el hecho de que es en tiempo presente, y así suces. Luego, las capas de atención de productos de puntos a escala implementan la fase de búsqueda y, finalmente, concatenamos todos los resultados y los proyectamos de nuevo al espacio original.

Keras incluye una capa `tf.keras.layers.MultiHeadAttention`, por lo que ahora tenemos todo lo que necesitamos para construir el resto del transformador. Comencemos con el codificador completo, que es exactamente como en la Figura 16-8, excepto que usamos una pila de dos bloques (`N = 2`) en lugar de seis, ya que no tenemos un conjunto de entrenamiento enorme, y agregamos un poco de abandono también:

In [ ]:
N = 2  # instead of 6
num_heads = 8
dropout_rate = 0.1
n_units = 128  # for the first dense layer in each feedforward block
encoder_pad_mask = tf.math.not_equal(encoder_input_ids, 0)[:, tf.newaxis]
Z = encoder_in
for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=encoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    Z = tf.keras.layers.Dropout(dropout_rate)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

Este código debería ser bastante sencillo, excepto por una cosa: el enmascaramiento. Al momento de escribir este artículo, la capa `MultiHeadAttention` no admite el enmascaramiento automático,⁠25 por lo que debemos manejarlo manualmente. ¿Cómo podemos hacer eso?

La capa `MultiHeadAttention` acepta un argumento de `attention_mask`, que es un tensor booleano de forma [tamaño de lote, longitud máxima de la consulta, longitud máxima del valor]: para cada token en cada secuencia de consulta, esta máscara indica qué tokens en la secuencia de valores correspondiente deben ser atendidos. . Queremos decirle a la capa `MultiHeadAttention` que ignore todos los tokens de relleno en los valores. Entonces, primero calculamos la máscara de relleno usando `tf.math.not_equal(encoder_input_ids, 0)`. Esto devuelve un tensor booleano de forma [tamaño de lote, longitud máxima de secuencia]. Luego insertamos un segundo eje usando `[:, tf.newaxis]`, para obtener una máscara de forma [tamaño de lote, 1, longitud máxima de secuencia]. Esto nos permite usar esta máscara como máscara de atención al llamar a la capa de Atención MultiHead: gracias a la transmisión, se usará la misma máscara para todos los tokens en cada consulta. De esta manera, los tokens de relleno en los valores se ignorarán correctamente.

Sin embargo, la capa calculará los resultados de cada token de consulta, incluidos los tokens de relleno. Necesitamos enmascarar las salidas que corresponden a estos tokens de relleno. Recuerde que usamos `mask_zero` en las capas de `Embedding` y configuramos support_masking en True en la capa PositionalEncoding, por lo que la máscara automática se propagó hasta las entradas de la capa MultiHeadAttention (encoder_in). Podemos usar esto a nuestro favor en la conexión de omisión: de hecho, la capa Agregar admite el enmascaramiento automático, por lo que cuando agregamos Z y omitimos (que inicialmente es igual a encoder_in), las salidas se enmascaran automáticamente y correctamente. ¡Ay! El enmascaramiento requirió mucha más explicación que el código.

¡Ahora vamos al decodificador! Una vez más, el enmascaramiento va a ser la única parte complicada, así que comencemos con eso. La primera capa de atención de cabeza múltiple es una capa de autoatención, como en el codificador, pero es una capa de atención de cabeza múltiple enmascarada, lo que significa que es causal: debería ignorar todos los tokens en el futuro. Por lo tanto, necesitamos dos máscaras: una máscara de acolchado y una máscara causal. Vamos a crearlos:

In [ ]:
decoder_pad_mask = tf.math.not_equal(decoder_input_ids, 0)[:, tf.newaxis]
causal_mask = tf.linalg.band_part(  # creates a lower triangular matrix
    tf.ones((batch_max_len_dec, batch_max_len_dec), tf.bool), -1, 0)

La máscara de relleno es exactamente igual a la que creamos para el codificador, excepto que se basa en las entradas del decodificador en lugar de las del codificador. La máscara causal se crea usando la función `tf.linalg.band_part()`, que toma un tensor y devuelve una copia con todos los valores fuera de una banda diagonal establecidos en cero. Con estos argumentos, obtenemos una matriz cuadrada de tamaño `batch_max_len_dec` (la longitud máxima de las secuencias de entrada en el lote), con 1 en el triángulo inferior izquierdo y 0 en la parte superior derecha. Si usamos esta máscara como máscara de atención, obtendremos exactamente lo que queremos: el primer token de consulta solo atenderá al primer token de valor, el segundo solo atenderá a los dos primeros, el tercero solo atenderá a los tres primeros , etcétera. En otras palabras, los tokens de consulta no pueden atender a ningún token de valor en el futuro.

Ahora construimos el decodificador:

In [ ]:
encoder_outputs = Z  # let's save the encoder's final outputs
Z = decoder_in  # the decoder starts with its own inputs
for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=causal_mask & decoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=encoder_outputs, attention_mask=encoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

Para la primera capa de atención, utilizamos `causal_mask & decoder_pad_mask` para enmascarar tanto los tokens de relleno como los tokens futuros. La máscara causal solo tiene dos dimensiones: le falta la dimensión del lote, pero eso está bien ya que la transmisión garantiza que se copie en todas las instancias del lote.

Para la segunda capa de atención, no hay nada especial. Lo único que hay que tener en cuenta es que estamos usando `encoder_pad_mask`, no `decoder_pad_mask`, porque esta capa de atención utiliza las salidas finales del codificador como sus valores.

Ya casi hemos terminado. Solo tenemos que añadir la capa de salida final, crear el modelo, compilarlo y entrenarlo:

In [ ]:
Y_proba = tf.keras.layers.Dense(vocab_size, activation="softmax")(Z)
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

¡Felicidades! Has construido un transformador completo desde cero y lo has entrenado para la traducción automática. ¡Esto está bastante avanzado!

#### TIP

El equipo de Keras ha creado un nuevo proyecto de PNL de Keras, que incluye una API para construir un transformador más fácilmente. También puede estar interesado en el nuevo proyecto de CV de Keras para la visión por ordenador.

#### -------------------------------------------------------------------------------------

Pero el campo no se detuvo allí. Ahora exploremos algunos de los avances recientes.

# Una avalancha de modelos de transformadores

El año 2018 ha sido llamado el "momento de ImageNet para la PNL". Desde entonces, el progreso ha sido asombroso, con arquitecturas cada vez más grandes basadas en transformadores entrenadas en inmensos conjuntos de datos.

En primer lugar, el documento GPT⁠ de Alec Radford y otros investigadores de OpenAI demostró una vez más la eficacia del preentrenamiento no supervisado, como los documentos ELMo y ULMFiT anteriores, pero esta vez utilizando una arquitectura similar a un transformador. Los autores preentrenaron una arquitectura grande pero bastante simple compuesta por una pila de 12 módulos de transformador que utilizan solo capas de atención de múltiples cabezas enmascaradas, como en el decodificador del transformador original. Lo entrenaron en un conjunto de datos muy grande, utilizando la misma técnica autorregresiva que usamos para nuestro char-RNN de Shakespeare: solo predecir el siguiente token. Esta es una forma de aprendizaje autosupervisado. Luego lo ajustaron a varias tareas de lenguaje, utilizando solo adaptaciones menores para cada tarea. Las tareas eran bastante diversas: incluían la clasificación del texto, la implicación (ya sea que la oración A imponga, involucre o implique la oración B como una consecuencia necesaria), ⁠similitud (por ejemplo, "El buen tiempo hoy" es muy similar a "Hace sol") y la respuesta a preguntas (dados unos pocos párrafos de texto que dan algo de contexto, el modelo debe responder a algunas preguntas de opción múltiple).

Luego salió el documento BERT⁠ de Google: también demostró la eficacia del preentrenamiento autosupervisado en un cuerpo grande, utilizando una arquitectura similar a GPT, pero solo con capas de atención de múltiples cabezas no enmascaradas, como en el codificador del transformador original. Esto significa que el modelo es naturalmente bidireccional; de ahí la B en BERT (representaciones de codificador bidireccionales de transformadores). Lo más importante es que los autores propusieron dos tareas de preentrenamiento que explican la mayor parte de la fuerza del modelo:

- **Modelo de lenguaje enmascarado (MLM)**

    Cada palabra de una oración tiene una probabilidad del 15% de estar enmascarada, y el modelo está entrenado para predecir las palabras enmascaradas. Por ejemplo, si la frase original es "Ella se divirtió en la fiesta de cumpleaños", entonces se le puede dar a la modelo la frase "Ella <máscara> se divierte en la fiesta <máscara>" y debe predecir las palabras "tenía" y "cumpleaños" (los otros resultados serán ignorados). Para ser más precisos, cada palabra seleccionada tiene un 80% de probabilidad de ser enmascarada, un 10% de probabilidad de ser reemplazada por una palabra aleatoria (para reducir la discrepancia entre el preentrenamiento y el ajuste fino, ya que el modelo no verá tokens <máscara> durante el ajuste fino), y un 10% de probabilidad de quedar solo (para sesgar el modelo hacia la respuesta correcta).

* **Predicción de la siguiente oración (NSP)**

    El modelo está entrenado para predecir si dos oraciones son consecutivas o no. Por ejemplo, debería predecir que "El perro duerme" y "Ronca fuerte" son oraciones consecutivas, mientras que "El perro duerme" y "La tierra orbita el Sol" no son consecutivas. Investigaciones posteriores mostraron que NSP no era tan importante como se pensaba inicialmente, por lo que se eliminó en la mayoría de las arquitecturas posteriores.

El modelo está entrenado en estas dos tareas simultáneamente (ver Figura 16-11). Para la tarea NSP, los autores insertaron un token de clase (<CLS>) al comienzo de cada entrada, y el token de salida correspondiente representa la predicción del modelo: la oración B sigue a la frase A, o no lo hace. Las dos oraciones de entrada están concatenadas, separadas solo por un token de separación especial (<SEP>), y se alimentan como entrada al modelo. Para ayudar al modelo a saber a qué oración pertenece cada token de entrada, se agrega una incrustación de segmento encima de las incrustaciones posicionales de cada token: solo hay dos incrustaciones de segmento posibles, una para la oración A y otra para la oración B. Para la tarea de MLM, algunas palabras de entrada están enmascaradas (como acabamos de ver) y el modelo intenta predecir cuáles eran esas palabras. La pérdida solo se calcula en la predicción de NSP y en los tokens enmascarados, no en los desenmascarados.


![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1611.png)
    
(_Figura 16-11. Proceso de formación y ajuste fino de BERT⁠_)
    
Después de esta fase de preentrenamiento no supervisada en un corpus muy grande de texto, el modelo se afina en muchas tareas diferentes, cambiando muy poco para cada tarea. Por ejemplo, para la clasificación de texto, como el análisis de sentimientos, se ignoran todos los tokens de salida, excepto el primero, que corresponde al token de clase, y una nueva capa de salida reemplaza a la anterior, que era solo una capa de clasificación binaria para NSP.

En febrero de 2019, solo unos meses después de que se publicara BERT, Alec Radford, Jeffrey Wu y otros investigadores de OpenAI publicaron el documento GPT-2,⁠31, que proponía una arquitectura muy similar a la de GPT, pero aún más grande (¡con más de 1.500 millones de parámetros!). Los investigadores demostraron que el nuevo y mejorado modelo GPT podría realizar un aprendizaje de tiro cero (ZSL), lo que significa que podría lograr un buen rendimiento en muchas tareas sin ningún ajuste fino. Esto fue solo el comienzo de una carrera hacia modelos cada vez más grandes: los Switch Transformers⁠32 de Google (introducidos en enero de 2021) utilizaron 1 billón de parámetros, y pronto salieron modelos mucho más grandes, como el modelo Wu Dao 2.0 de la Academia de Inteligencia Artificial de Beijing (BAII), anunciado en junio de 2021.

Una consecuencia desafortunada de esta tendencia hacia modelos gigantescos es que solo las organizaciones bien financiadas pueden permitirse el lujo de entrenar a tales modelos: puede costar fácilmente cientos de miles de dólares o más. Y la energía necesaria para entrenar un solo modelo corresponde al consumo de electricidad de un hogar estadounidense durante varios años; no es respetuoso con el medio ambiente en absoluto. Muchos de estos modelos son demasiado grandes para ser utilizados en hardware normal: no cabrían en la RAM y serían terriblemente lentos. Por último, algunos son tan costosos que no se publican públicamente.

Afortunadamente, los investigadores ingeniosos están encontrando nuevas formas de reducir el tamaño de los transformadores y hacerlos más eficientes en cuanto a los datos. Por ejemplo, el modelo DistilBERT,⁠33 introducido en octubre de 2019 por Victor Sanh et al. de Hugging Face, es un modelo de transformador pequeño y rápido basado en BERT. Está disponible en el excelente hub de modelo de Hugging Face, junto con miles de otros; verás un ejemplo más adelante en este capítulo.

DistilBERT se entrenó utilizando la destilación (de ahí el nombre): esto significa transferir el conocimiento de un modelo de profesor a uno de estudiante, que suele ser mucho más pequeño que el modelo de profesor. Por lo general, esto se hace utilizando las probabilidades predichas por el profesor para cada instancia de entrenamiento como objetivos para el estudiante. ¡Sorprendentemente, la destilación a menudo funciona mejor que entrenar al estudiante desde cero en el mismo conjunto de datos que el profesor! De hecho, el estudiante se beneficia de las etiquetas más matizadas del profesor.

Muchas más arquitecturas de transformadores salieron después de BERT, casi mensualmente, a menudo mejorando el estado del arte en todas las tareas de PNL: XLNet (junio de 2019), RoBERTa (julio de 2019), StructBERT (agosto de 2019), ALBERT (septiembre de 2019), T5 (octubre de 2019), ELECTRA (marzo de 2020), GPT3 (mayo de 2020), DeBERTa (junio de 2020), Switch Transformers (enero de 2021), Wu Dao 2.0 (junio de 2021), Gopher (diciembre de 2021), GPT-NeoX-20B (febrero de 2022), Chinchilla (marzo de 2022), OPT (mayo de 2022), y la lista sigue y sigue. Cada uno de estos modelos trajo nuevas ideas y técnicas,34 pero me gusta especialmente el papel T5⁠35 de los investigadores de Google: enmarca todas las tareas de PNL como texto a texto, utilizando un transformador de codificador-decodificador. Por ejemplo, para traducir "Me gusta el fútbol" al español, puedes llamar al modelo con la frase de entrada "traducir inglés al español: me gusta el fútbol" y muestra "me gusta el fútbol". Para resumir un párrafo, solo tienes que introducir "resumir:" seguido del párrafo, y muestra el resumen. Para la clasificación, solo necesita cambiar el prefijo a "clasificar:" y el modelo muestra el nombre de la clase, como texto. Esto simplifica el uso del modelo, y también hace posible preentrenarlo en aún más tareas.

Por último, pero no menos importante, en abril de 2022, los investigadores de Google utilizaron una nueva plataforma de capacitación a gran escala llamada Pathways (que discutiremos brevemente en el capítulo 19) para entrenar un enorme modelo de lenguaje llamado Pathways Language Model (PaLM), ⁠36 con la friolera de 540 mil millones de parámetros, utilizando más de 6.000 TPU. Aparte de su increíble tamaño, este modelo es un transformador estándar, que utiliza solo decodificadores (es decir, con capas de atención de varias cabezas enmascaradas), con solo unos pocos ajustes (consulte el documento para más detalles). Este modelo logró un rendimiento increíble en todo tipo de tareas de PNL, particularmente en la comprensión del lenguaje natural (NLU). Es capaz de realizar hazañas impresionantes, como explicar chistes, dar respuestas detalladas paso a paso a las preguntas e incluso codificar. Esto se debe en parte al tamaño del modelo, pero también gracias a una técnica llamada Chain of thought prompting,⁠37 que fue introducida un par de meses antes por otro equipo de investigadores de Google.

En las tareas de respuesta a preguntas, la solicitud regular suele incluir algunos ejemplos de preguntas y respuestas, como: "P: Roger tiene 5 pelotas de tenis. Compra 2 latas más de pelotas de tenis. Cada lata tiene 3 pelotas de tenis. ¿Cuántas pelotas de tenis tiene ahora? A: 11". El aviso continúa con la pregunta real, como "P: John cuida de 10 perros. Cada perro tarda 0,5 horas al día en pasear y ocuparse de sus asuntos. ¿Cuántas horas a la semana pasa cuidando a los perros? A:", y el trabajo del modelo es añadir la respuesta: en este caso, "35".

Pero con la cadena de pensamiento, las respuestas de ejemplo incluyen todos los pasos de razonamiento que conducen a la conclusión. Por ejemplo, en lugar de "A: 11", el mensaje contiene "A: Roger comenzó con 5 bolas. 2 latas de 3 pelotas de tenis cada una son 6 pelotas de tenis. 5 + 6 = 11. Esto anima al modelo a dar una respuesta detallada a la pregunta real, como "John cuida de 10 perros". Cada perro tarda 0,5 horas al día en pasear y ocuparse de sus asuntos. Así que eso es 10 × .5 = 5 horas al día. 5 horas al día × 7 días a la semana = 35 horas a la semana. La respuesta es 35 horas a la semana". ¡Este es un ejemplo real del periódico!

El modelo no solo da la respuesta correcta con mucha más frecuencia que usar indicaciones regulares, estamos alentando al modelo a pensar las cosas, sino que también proporciona todos los pasos de razonamiento, que pueden ser útiles para comprender mejor la justificación detrás de la respuesta de un modelo.

Los transformadores se han hecho cargo de la PNL, pero no se detuvieron ahí: pronto también se expandieron a la visión por ordenador.
    
    
# Transformadores de visión

Una de las primeras aplicaciones de los mecanismos de atención más allá de NMT fue la generación de subtítulos de imagen utilizando la atención visual: una red neuronal convolucional primero procesa la imagen y genera algunos mapas de características, luego un decodificador RNN equipado con un mecanismo de atención genera el pie de foto, una palabra a la vez.

En cada paso de tiempo del decodificador (es decir, cada palabra), el decodificador utiliza el modelo de atención para centrarse en la parte correcta de la imagen. Por ejemplo, en la Figura 16-12, el modelo generó el título "Una mujer está lanzando un frisbee en un parque", y se puede ver en qué parte de la imagen de entrada el decodificador centró su atención cuando estaba a punto de emitir la palabra "frisbee": claramente, la mayor parte de su atención se centró en el frisbee.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1612.png)

(_Figura 16-12. Atención visual: una imagen de entrada (izquierda) y el enfoque del modelo antes de producir la palabra "frisbee" (derecha)⁠_)

#### EXPLICABILIDAD

Un beneficio adicional de los mecanismos de atención es que facilitan la comprensión de lo que llevó al modelo a producir su producción. Esto se llama explicabilidad. Puede ser especialmente útil cuando el modelo comete un error: por ejemplo, si una imagen de un perro caminando en la nieve está etiquetada como "un lobo caminando en la nieve", entonces puedes volver atrás y comprobar en qué se centró el modelo cuando emite la palabra "lobo". Es posible que descubras que estaba prestando atención no solo al perro, sino también a la nieve, insinuando una posible explicación: tal vez la forma en que el modelo aprendió a distinguir a los perros de los lobos es comprobando si hay o no mucha nieve alrededor. Luego puedes arreglar esto entrenando al modelo con más imágenes de lobos sin nieve y perros con nieve. Este ejemplo proviene de un gran artículo de 2016⁠ de Marco Tulio Ribeiro et al. que utiliza un enfoque diferente para la explicabilidad: aprender un modelo interpretable localmente en torno a la predicción de un clasificador.

En algunas aplicaciones, la explicabilidad no es solo una herramienta para depurar un modelo; puede ser un requisito legal: piense en un sistema que decida si debe o no concederle un préstamo.

#### --------------------------------------------------------------------------------------

Cuando los transformadores salieron en 2017 y la gente comenzó a experimentar con ellos más allá de la PNL, se usaron por primera vez junto con las CNN, sin reemplazarlos. En su lugar, los transformadores se utilizaban generalmente para reemplazar los RNN, por ejemplo, en los modelos de subtítulos de imágenes. Transformers se volvió un poco más visual en un documento de 2020 de los investigadores de Facebook, que propuso una arquitectura híbrida de CNN-transformador para la detección de objetos. Una vez más, la CNN primero procesa las imágenes de entrada y emite un conjunto de mapas de características, luego estos mapas de características se convierten en secuencias y se alimentan a un transformador, que genera predicciones de cajas delimitadoras. Pero, de nuevo, la mayor parte del trabajo visual todavía lo realiza la CNN.

Luego, en octubre de 2020, un equipo de investigadores de Google publicó un documento⁠ que introdujo un modelo de visión totalmente basado en transformadores, llamado transformador de visión (ViT). La idea es sorprendentemente simple: simplemente corta la imagen en pequeños cuadrados de 16 × 16 y trata la secuencia de cuadrados como si fuera una secuencia de representaciones de palabras. Para ser más precisos, los cuadrados se aplanan primero en vectores de 16 × 16 × 3 = 768 dimensiones, el 3 es para los canales de color RGB, luego estos vectores pasan por una capa lineal que los transforma pero conserva su dimensionalidad. La secuencia resultante de vectores se puede tratar como una secuencia de incrustaciones de palabras: esto significa agregar incrustaciones posicionales y pasar el resultado al transformador. ¡Eso es todo! Este modelo superó al estado del arte en la clasificación de imágenes de ImageNet, pero para ser justos, los autores tuvieron que usar más de 300 millones de imágenes adicionales para la capacitación. Esto tiene sentido, ya que los transformadores no tienen tantos sesgos inductivos como las redes neuronales de convolución, por lo que necesitan datos adicionales solo para aprender cosas que las CNN asumen implícitamente.

#### NOTA

Un sesgo inductivo es una suposición implícita hecha por el modelo, debido a su arquitectura. Por ejemplo, los modelos lineales asumen implícitamente que los datos son, bueno, lineales. Las CNN asumen implícitamente que los patrones aprendidos en un lugar probablemente también serán útiles en otros lugares. Los RNN asumen implícitamente que las entradas están ordenadas y que los tokens recientes son más importantes que los más antiguos. Cuantos más sesgos inductivos tenga un modelo, suponiendo que sean correctos, menos datos de entrenamiento requerirá el modelo. Pero si las suposiciones implícitas son incorrectas, entonces el modelo puede funcionar mal, incluso si está entrenado en un conjunto de datos grande.

#### -------------------------------------------------------------------------------------

Solo dos meses después, un equipo de investigadores de Facebook publicó un documento⁠43 que introdujo transformadores de imagen eficientes en datos (DeiT). Su modelo logró resultados competitivos en ImageNet sin requerir ningún dato adicional para la capacitación. La arquitectura del modelo es prácticamente la misma que la ViT original, pero los autores utilizaron una técnica de destilación para transferir el conocimiento de los modelos de CNN de última generación a su modelo.

Luego, en marzo de 2021, DeepMind publicó un importante documento⁠44 que introdujo la arquitectura Perceiver. Es un transformador multimodal, lo que significa que puedes alimentarlo con texto, imágenes, audio o prácticamente cualquier otra modalidad. Hasta entonces, los transformadores se habían restringido a secuencias bastante cortas debido al rendimiento y al cuello de botella de la RAM en las capas de atención. Esto excluyó modalidades como el audio o el vídeo, y obligó a los investigadores a tratar las imágenes como secuencias de parches, en lugar de secuencias de píxeles. El cuello de botella se debe a la autoatención, donde cada token debe atender a cada otro token: si la secuencia de entrada tiene tokens M, entonces la capa de atención debe calcular una matriz M × M, que puede ser enorme si M es muy grande. El Perceptor resuelve este problema mejorando gradualmente una representación latente bastante corta de las entradas, compuesta por N tokens, normalmente solo unos pocos cientos. (La palabra latente significa oculto o interno). El modelo utiliza solo capas de atención cruzada, alimentándolas con la representación latente como las consultas y las entradas (posiblemente grandes) como los valores. Esto solo requiere calcular una matriz M × N, por lo que la complejidad computacional es lineal con respecto a M, en lugar de cuadrática. Después de pasar por varias capas de atención cruzada, si todo va bien, la representación latente termina capturando todo lo que importa en las entradas. Los autores también sugirieron compartir los pesos entre capas consecutivas de atención cruzada: si haces eso, entonces el Perceptor se convierte efectivamente en un RNN. De hecho, las capas de atención cruzada compartida se pueden ver como la misma celda de memoria en diferentes pasos de tiempo, y la representación latente corresponde al vector de contexto de la celda. Las mismas entradas se alimentan repetidamente a la celda de memoria en cada paso. ¡Parece que los RNN no están muertos después de todo!

Solo un mes después, Mathilde Caron et al. presentaron DINO,45, un impresionante transformador de visión entrenado completamente sin etiquetas, utilizando autosupervisión y capaz de segmentación semántica de alta precisión. El modelo se duplica durante la formación, con una red actuando como profesor y la otra actuando como estudiante. El descenso del gradiente solo afecta al estudiante, mientras que los pesos del profesor son solo un promedio móvil exponencial de los pesos del estudiante. El estudiante está capacitado para coincidir con las predicciones del profesor: ya que son casi el mismo modelo, esto se llama autodestilación. En cada paso de entrenamiento, las imágenes de entrada se aumentan de diferentes maneras para el profesor y el estudiante, por lo que no ven exactamente la misma imagen, pero sus predicciones deben coincidir. Esto los obliga a llegar a representaciones de alto nivel. Para evitar el colapso del modo, donde tanto el estudiante como el profesor siempre emitirían lo mismo, ignorando por completo las entradas, DINO realiza un seguimiento de un promedio móvil de los resultados del profesor, y ajusta las predicciones del profesor para garantizar que permanezcan centradas en cero, en promedio. DINO también obliga al profesor a tener una gran confianza en sus predicciones: esto se llama afilar. Juntas, estas técnicas preservan la diversidad en los resultados del profesor.

En un artículo de 2021, ⁠46 investigadores de Google mostraron cómo escalar los ViT hacia arriba o hacia abajo, dependiendo de la cantidad de datos. Se las arreglaron para crear un enorme modelo de 2 mil millones de parámetros que alcanzó más del 90,4 % de precisión superior a 1 en ImageNet. Por el contrario, también entrenaron un modelo reducido que alcanzó más del 84,8 % de precisión de la parte superior 1 en ImageNet, utilizando solo 10.000 imágenes: ¡eso es solo 10 imágenes por clase!

Y el progreso en los transformadores visuales ha continuado de manera constante hasta el día de hoy. Por ejemplo, en marzo de 2022, un documento⁠47 de Mitchell Wortsman et al. demostró que es posible primero entrenar a varios transformadores, luego promediar sus pesos para crear un modelo nuevo y mejorado. Esto es similar a un conjunto (ver Capítulo 7), excepto que solo hay un modelo al final, lo que significa que no hay penalización de tiempo de inferencia.

La última tendencia en transformadores consiste en la construcción de grandes modelos multimodales, a menudo capaces de aprendizaje de tiro cero o poco disparo. Por ejemplo, el documento CLIP de 2021 de OpenAI⁠48 propuso un gran modelo de transformador preentrenado para hacer coincidir los subtítulos con las imágenes: esta tarea le permite aprender excelentes representaciones de imágenes, y el modelo se puede utilizar directamente para tareas como la clasificación de imágenes utilizando indicaciones de texto simples como "una foto de un gato". Poco después, OpenAI anunció DALL·E,⁠49 capaz de generar imágenes increíbles basadas en mensajes de texto. El DALL·E 2,⁠50 que genera imágenes de mayor calidad utilizando un modelo de difusión (ver Capítulo 17).

En abril de 2022, DeepMind publicó el documento Flamingo,⁠51, que introdujo una familia de modelos preentrenados en una amplia variedad de tareas a través de múltiples modalidades, incluyendo texto, imágenes y vídeos. Un solo modelo se puede utilizar en tareas muy diferentes, como la respuesta a preguntas, los subtítulos de imágenes y más. Poco después, en mayo de 2022, DeepMind introdujo GATO,⁠52, un modelo multimodal que se puede utilizar como política para un agente de aprendizaje de refuerzo (RL se introducirá en el capítulo 18). El mismo transformador puede chatear contigo, subtitular imágenes, jugar a juegos de Atari, controlar brazos robóticos (simulados) y más, todo con "solo" 1.200 millones de parámetros. ¡Y la aventura continúa!

#### NOTA

Estos asombrosos avances han llevado a algunos investigadores a afirmar que la IA a nivel humano está cerca, que "la escala es todo lo que necesitas", y que algunos de estos modelos pueden ser "ligeramente conscientes". Otros señalan que, a pesar del increíble progreso, estos modelos todavía carecen de la fiabilidad y adaptabilidad de la inteligencia humana, nuestra capacidad de razonar simbólicamente, de generalizar en base a un solo ejemplo y más.

#### -------------------------------------------------------------------------------------

Como puedes ver, ¡los transformadores están en todas partes! Y la buena noticia es que generalmente no tendrá que implementar transformadores usted mismo, ya que muchos modelos excelentes preentrenados están fácilmente disponibles para su descarga a través de TensorFlow Hub o el hub de modelos de Hugging Face. Ya has visto cómo usar un modelo de TF Hub, así que cerremos este capítulo echando un vistazo rápido al ecosistema de Hugging Face.


# Biblioteca de Transformers de Hugging Face

Hoy en día es imposible hablar de los transformadores sin mencionar Hugging Face, una empresa de IA que ha construido todo un ecosistema de herramientas de código abierto fáciles de usar para la PNL, la visión y más allá. El componente central de su ecosistema es la biblioteca de Transformers, que le permite descargar fácilmente un modelo preentrenado, incluido su tokenizador correspondiente, y luego ajustarlo en su propio conjunto de datos, si es necesario. Además, la biblioteca es compatible con TensorFlow, PyTorch y JAX (con la biblioteca Flax).

La forma más sencilla de usar la biblioteca Transformers es usar la función `transformers.pipe⁠line()`: solo tienes que especificar qué tarea quieres, como el análisis de sentimientos, y descarga un modelo preentrenado predeterminado, listo para ser utilizado, realmente no podría ser más simple:

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")  # many other tasks are available
result = classifier("The actors were very convincing".)

El resultado es una lista de Python que contiene un diccionario por texto de entrada:

In [ ]:
result

'''
[{'label': 'POSITIVE', 'score': 0.9998071789741516}]
'''

En este ejemplo, el modelo encontró correctamente que la oración es positiva, con alrededor del 99,98 % de confianza. Por supuesto, también puedes pasar un lote de oraciones al modelo:

In [ ]:
classifier(["I am from India.", "I am from Iraq."])

'''
[{'label': 'POSITIVE', 'score': 0.9896161556243896},
 {'label': 'NEGATIVE', 'score': 0.9811071157455444}]
 '''

#### SESGO Y EQUIDAD

Como sugiere la producción, este clasificador específico ama a los indios, pero está severamente sesgado contra los iraquíes. Puedes probar este código con tu propio país o ciudad. Tal sesgo indeseable generalmente proviene en gran parte de los propios datos de entrenamiento: en este caso, había muchas frases negativas relacionadas con las guerras en Irak en los datos de entrenamiento. Este sesgo se amplificó durante el proceso de ajuste fino, ya que el modelo se vio obligado a elegir entre solo dos clases: positiva o negativa. Si agregas una clase neutral al ajustar, entonces el sesgo del país desaparece en su mayor parte. Pero los datos de entrenamiento no son la única fuente de sesgo: la arquitectura del modelo, el tipo de pérdida o regularización utilizada para el entrenamiento, el optimizador; todo esto puede afectar lo que el modelo termina aprendiendo. Incluso un modelo en su mayoría imparcial se puede utilizar de una manera sesgada, al igual que las preguntas de la encuesta pueden ser sesgadas.

Comprender el sesgo en la IA y mitigar sus efectos negativos sigue siendo un área de investigación activa, pero una cosa es cierta: debe hacer una pausa y pensar antes de apresurarse a desplegar un modelo en la producción. Pregúntate cómo el modelo podría hacer daño, incluso indirectamente. Por ejemplo, si las predicciones del modelo se utilizan para decidir si dar o no un préstamo a alguien, el proceso debe ser justo. Por lo tanto, asegúrese de evaluar el rendimiento del modelo no solo en promedio en todo el conjunto de pruebas, sino también en varios subconjuntos: por ejemplo, puede encontrar que, aunque el modelo funciona muy bien en promedio, su rendimiento es abismal para algunas categorías de personas. También es posible que desee ejecutar pruebas contrafácticas: por ejemplo, es posible que desee comprobar que las predicciones del modelo no cambian cuando simplemente cambia el género de alguien.

Si el modelo funciona bien en promedio, es tentador empujarlo a la producción y pasar a otra cosa, especialmente si es solo un componente de un sistema mucho más grande. Pero en general, si no solucionas tales problemas, nadie más lo hará, y tu modelo puede terminar haciendo más daño que bien. La solución depende del problema: puede requerir reequilibrar el conjunto de datos, ajustar un conjunto de datos diferente, cambiar a otro modelo preentrenado, ajustar la arquitectura o los hiperparámetros del modelo, etc.

#### -------------------------------------------------------------------------------------

La función `pipeline()` utiliza el modelo predeterminado para la tarea dada. Por ejemplo, para tareas de clasificación de texto como el análisis de sentimientos, al momento de escribir este artículo, el valor predeterminado es `distilbert-base-uncased-finetuned-sst-2-english`: un modelo DistilBERT con un tokenizador sin caja, entrenado en Wikipedia en inglés y un corpus. de libros en inglés y afinado en la tarea Stanford Sentiment Treebank v2 (SST 2). También es posible especificar manualmente un modelo diferente. Por ejemplo, podría utilizar un modelo DistilBERT ajustado en la tarea Multi-Genre Natural Language Inference (MultiNLI), que clasifica dos oraciones en tres clases: contradicción, neutral o vinculación. Aquí es cómo:

In [ ]:
model_name = "huggingface/distilbert-base-uncased-finetuned-mnli"
classifier_mnli = pipeline("text-classification", model=model_name)
classifier_mnli("She loves me. [SEP] She loves me not.")

'''
[{'label': 'contradiction', 'score': 0.9790192246437073}]
'''

#### TIP

Puedes encontrar los modelos disponibles en https://huggingface.co/models, y la lista de tareas en https://huggingface.co/tasks.

#### --------------------------------------------------------------------------------------

La API de canalización es muy simple y conveniente, pero a veces necesitarás más control. Para tales casos, la biblioteca Transformers proporciona muchas clases, incluidos todo tipo de tokenizadores, modelos, configuraciones, devoluciones de llamadas y mucho más. Por ejemplo, carguemos el mismo modelo DistilBERT, junto con su tokenizador correspondiente, usando las clases `TFAutoModelForSequenceClassification` y `AutoTokenizer`:

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

A continuación, tokenicemos un par de pares de oraciones. En este código, activamos el relleno y especificamos que queremos tensores de TensorFlow en lugar de listas de Python:

In [ ]:
token_ids = tokenizer(["I like soccer. [SEP] We all love soccer!",
                       "Joe lived for a very long time. [SEP] Joe is old."],
                      padding=True, return_tensors="tf")

#### PROPINA

Instead of passing "Sentence 1 [SEP] Sentence 2" to the tokenizer, you can equivalently pass it a tuple: ("Sentence 1", "Sentence 2").

#### --------------------------------------------------------------------------------------

La salida es una instancia similar a un diccionario de la clase `BatchEncoding`, que contiene las secuencias de ID de token, así como una máscara que contiene 0 para los tokens de relleno:

In [ ]:
token_ids

'''
{'input_ids': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[ 101, 1045, 2066, 4715, 1012,  102, 2057, 2035, 2293, 4715,  999,
         102,    0,    0,    0],
       [ 101, 3533, 2973, 2005, 1037, 2200, 2146, 2051, 1012,  102, 3533,
        2003, 2214, 1012,  102]], dtype=int32)>,
 'attention_mask': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}
'''

Si configura `return_token_type_ids=True` al llamar al tokenizador, también obtendrá un tensor adicional que indica a qué oración pertenece cada token. Algunos modelos lo necesitan, pero no DistilBERT.

A continuación, podemos pasar directamente este objeto `BatchEncoding` al modelo; devuelve un objeto `TFSequenceClassifierOutput` que contiene sus logits de clase predichos:

In [ ]:
outputs = model(token_ids)
outputs

'''
TFSequenceClassifierOutput(loss=None, logits=[<tf.Tensor: [...] numpy=
array([[-2.1123817 ,  1.1786783 ,  1.4101017 ],
       [-0.01478387,  1.0962474 , -0.9919954 ]], dtype=float32)>], [...])
'''

Por último, podemos aplicar la función de activación softmax para convertir estos logits en probabilidades de clase y usar la función `argmax()` para predecir la clase con la mayor probabilidad para cada par de oraciones de entrada:

In [ ]:
Y_probas = tf.keras.activations.softmax(outputs.logits)
Y_probas

'''
<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.01619702, 0.43523544, 0.5485676 ],
       [0.08672056, 0.85204804, 0.06123142]], dtype=float32)>
'''

Y_pred = tf.argmax(Y_probas, axis=1)
Y_pred  # 0 = contradiction, 1 = entailment, 2 = neutral

'''
<tf.Tensor: shape=(2,), dtype=int64, numpy=array([2, 1])>
'''

En este ejemplo, el modelo clasifica correctamente el primer par de oraciones como neutral (el hecho de que me guste el fútbol no implica que todos los demás lo hagan) y el segundo par como una implicación (Joe debe ser bastante viejo).

Si desea ajustar este modelo en su propio conjunto de datos, puede entrenar el modelo como de costumbre con Keras, ya que es solo un modelo Keras normal con algunos métodos adicionales. Sin embargo, debido a que el modelo genera logits en lugar de probabilidades, debe usar la pérdida `tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)` en lugar de la pérdida habitual `"sparse_categorical_crossentropy"`. Además, el modelo no admite entradas de `BatchEncoding` durante el entrenamiento, por lo que debes usar su atributo de datos para obtener un diccionario normal:

In [ ]:
sentences = [("Sky is blue", "Sky is red"), ("I love her", "She loves me")]
X_train = tokenizer(sentences, padding=True, return_tensors="tf").data
y_train = tf.constant([0, 2])  # contradiction, neutral
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=2)

Hugging Face también ha construido una biblioteca de conjuntos de datos que puedes usar para descargar fácilmente un conjunto de datos estándar (como IMDb) o uno personalizado, y usarlo para ajustar tu modelo. Es similar a los conjuntos de datos de TensorFlow, pero también proporciona herramientas para realizar tareas comunes de preprocesamiento sobre la marcha, como el enmascaramiento. La lista de conjuntos de datos está disponible en https://huggingface.co/datasets.

Esto debería ayudarte a empezar con el ecosistema de Hugging Face. Para obtener más información, puede dirigirse a https://huggingface.co/docs para obtener la documentación, que incluye muchos cuadernos tutoriales, vídeos, la API completa y más. También te recomiendo que eches un vistazo al libro de O'Reilly Natural Language Processing with Transformers: Building Language Applications with Hugging Face de Lewis Tunstall, Leandro von Werra y Thomas Wolf, todos del equipo de Hugging Face.

En el próximo capítulo discutiremos cómo aprender representaciones profundas de una manera no supervisada usando autocodificadores, ¡y usaremos redes adversarias generativas para producir imágenes y más!


# Ejercicios

1. ¿Cuáles son los pros y los contras de usar un RNN con estado frente a un RNN sin estado?

2. ¿Por qué la gente usa RNN de codificador-decodificador en lugar de RNN de secuencia a secuencia simple para la traducción automática?

3. ¿Cómo puedes lidiar con las secuencias de entrada de longitud variable? ¿Qué pasa con las secuencias de salida de longitud variable?

4. ¿Qué es la búsqueda de vigas y por qué la usarías? ¿Qué herramienta puedes usar para implementarla?

5. ¿Qué es un mecanismo de atención? ¿Cómo ayuda?

6. ¿Cuál es la capa más importante en la arquitectura del transformador? ¿Cuál es su propósito?

7. ¿Cuándo necesitarías usar softmax muestreado?

8. Las gramáticas Reber incrustadas fueron utilizadas por Hochreiter y Schmidhuber en su artículo sobre LSTM. Son gramáticas artificiales que producen cadenas como "BPBTSXXVPSEPE". Echa un vistazo a la bonita introducción de Jenny Orr a este tema, luego elige una gramática Reber incrustada en particular (como la representada en la página de Orr), luego entrena un RNN para identificar si una cadena respeta esa gramática o no. Primero tendrás que escribir una función capaz de generar un lote de entrenamiento que contenga alrededor del 50 % de cadenas que respeten la gramática y el 50 % que no.

9. Entrena un modelo de codificador-decodificador que pueda convertir una cadena de fecha de un formato a otro (por ejemplo, del "22 de abril de 2019" al "2019-04-22").

10. Revisa el ejemplo en el sitio web de Keras para "Búsqueda de imágenes en lenguaje natural con un codificador dual". Aprenderás a construir un modelo capaz de representar tanto imágenes como texto dentro del mismo espacio de incrustación. Esto hace posible buscar imágenes utilizando un mensaje de texto, como en el modelo CLIP de OpenAI.

11. Use the Hugging Face Transformers library to download a pretrained language model capable of generating text (e.g., GPT), and try generating more convincing Shakespearean text. You will need to use the model’s generate() method—see Hugging Face’s documentation for more details.

Las soluciones a estos ejercicios están disponibles al final del cuaderno de este capítulo, en https://homl.info/colab3.

